In [1]:
import keras
import tensorflow as tf
import random, os, glob
from keras.layers import *
from keras.models import Sequential, Model
from keras import backend as K
import numpy as np
import re
from keras.utils.vis_utils import model_to_dot
import random
import matplotlib.pyplot as plt
from scipy.spatial import distance
from IPython.display import Image
import PIL
from keras.models import model_from_json
from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


In [2]:
batch_size = 256
num_classes = 6
epochs = 1000
img_rows, img_cols = 28, 28
label = ["circle", "square", "triangle", "horizontal", "vertical", "diagonal"]

In [3]:
def load_data():
    tr_path = './dataset/trains/'
    te_path = './dataset/tests/'
    va_path = './dataset/vals/'
    
    (tr_sets, tr_labels, _) = get_datasets_labels(tr_path)
    (te_sets, te_labels, test_labels_with_num) = get_datasets_labels(te_path)
    (va_sets, va_labels, _) = get_datasets_labels(va_path)
    
    return tr_sets, tr_labels, te_sets, te_labels, va_sets, va_labels, test_labels_with_num

In [4]:
def get_datasets_labels(path):
	files = os.listdir(path)
	labels_to_num = []
	labels_with_num = []
	sets = []
	for file in files:
		picture = PIL.Image.open(path + file).convert('L')
		lb = file.replace(re.findall("\d+", file)[0], '').replace('.png', '') 
		label_with_num = file.replace('.png', '') 
		
		labels_to_num.append(label.index(lb))
		labels_with_num.append(label_with_num)
		
		picture = np.array(picture)
		sets.append(picture)
	
	return np.array(sets), np.array(labels_to_num), np.array(labels_with_num)

In [5]:
(x_train, y_train, x_test, y_test, x_val, y_val, test_labels_with_num) = load_data()
y_test1 = y_test

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

input_shape = (img_rows, img_cols, 1)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.
x_val = x_val.astype('float32')/255.

In [6]:
dense_model = Sequential()
dense_model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
dense_model.add(Activation('relu'))
dense_model.add(MaxPooling2D((2, 2), padding='same'))

dense_model.add(Conv2D(128, (3, 3), padding='same'))
dense_model.add(Activation('relu'))
dense_model.add(MaxPooling2D((2, 2), padding='same'))

dense_model.add(Conv2D(64, (3, 3), padding='same'))
dense_model.add(Activation('relu'))
dense_model.add(MaxPooling2D((2, 2), padding='same'))

dense_model.add(Conv2D(64, (3, 3), padding='same'))
dense_model.add(Activation('relu'))
dense_model.add(MaxPooling2D((2, 2), padding='same'))

dense_model.add(Flatten())
dense_model.add(Dense(128, activation='relu'))
dense_model.add(Dropout(0.25))

dense_model.add(Dense(64, activation = 'relu'))
dense_model.add(Dense(num_classes))
dense_model.add(Activation('softmax'))

#dense_model.summary()

In [7]:
dense_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [8]:
dense_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, validation_data=(x_val, y_val))

score = dense_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0], '\tTest accuracy:', score[1])

Train on 2400 samples, validate on 1800 samples
Epoch 1/1000
2400/2400 [==============================] - 2s 717us/step - loss: 1.7935 - acc: 0.1633 - val_loss: 1.7911 - val_acc: 0.2039
Epoch 2/1000
2400/2400 [==============================] - 0s 57us/step - loss: 1.7909 - acc: 0.1746 - val_loss: 1.7916 - val_acc: 0.1667
Epoch 3/1000
2400/2400 [==============================] - 0s 57us/step - loss: 1.7916 - acc: 0.1817 - val_loss: 1.7896 - val_acc: 0.1667
Epoch 4/1000
2400/2400 [==============================] - 0s 59us/step - loss: 1.7867 - acc: 0.1954 - val_loss: 1.7782 - val_acc: 0.1856
Epoch 5/1000
2400/2400 [==============================] - 0s 66us/step - loss: 1.7751 - acc: 0.2050 - val_loss: 1.8814 - val_acc: 0.1667
Epoch 6/1000
2400/2400 [==============================] - 0s 64us/step - loss: 1.7788 - acc: 0.2487 - val_loss: 1.7526 - val_acc: 0.2961
Epoch 7/1000
2400/2400 [==============================] - 0s 75us/step - loss: 1.7591 - acc: 0.2525 - val_loss: 1.7954 - val_acc:

2400/2400 [==============================] - 0s 74us/step - loss: 0.0070 - acc: 0.9979 - val_loss: 0.0667 - val_acc: 0.9861
Epoch 61/1000
2400/2400 [==============================] - 0s 75us/step - loss: 0.0041 - acc: 0.9996 - val_loss: 0.0601 - val_acc: 0.9844
Epoch 62/1000
2400/2400 [==============================] - 0s 63us/step - loss: 0.0715 - acc: 0.9758 - val_loss: 0.2256 - val_acc: 0.9383
Epoch 63/1000
2400/2400 [==============================] - 0s 61us/step - loss: 0.0267 - acc: 0.9929 - val_loss: 0.0589 - val_acc: 0.9861
Epoch 64/1000
2400/2400 [==============================] - 0s 70us/step - loss: 0.0050 - acc: 0.9992 - val_loss: 0.0618 - val_acc: 0.9833
Epoch 65/1000
2400/2400 [==============================] - 0s 83us/step - loss: 0.0046 - acc: 1.0000 - val_loss: 0.0617 - val_acc: 0.9856
Epoch 66/1000
2400/2400 [==============================] - 0s 88us/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 0.9867
Epoch 67/1000
2400/2400 [=======================

Epoch 119/1000
2400/2400 [==============================] - 0s 60us/step - loss: 4.8081e-04 - acc: 1.0000 - val_loss: 0.0720 - val_acc: 0.9850
Epoch 120/1000
2400/2400 [==============================] - 0s 56us/step - loss: 0.0886 - acc: 0.9800 - val_loss: 0.4762 - val_acc: 0.8783
Epoch 121/1000
2400/2400 [==============================] - 0s 69us/step - loss: 0.1198 - acc: 0.9642 - val_loss: 0.0629 - val_acc: 0.9828
Epoch 122/1000
2400/2400 [==============================] - 0s 71us/step - loss: 0.0031 - acc: 0.9996 - val_loss: 0.0555 - val_acc: 0.9856
Epoch 123/1000
2400/2400 [==============================] - 0s 74us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0569 - val_acc: 0.9850
Epoch 124/1000
2400/2400 [==============================] - 0s 71us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 0.9850
Epoch 125/1000
2400/2400 [==============================] - 0s 73us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0628 - val_acc: 0.9856
Epoch 126/1000
2400/240

2400/2400 [==============================] - 0s 72us/step - loss: 1.8726e-04 - acc: 1.0000 - val_loss: 0.0669 - val_acc: 0.9861
Epoch 177/1000
2400/2400 [==============================] - 0s 73us/step - loss: 1.0336e-04 - acc: 1.0000 - val_loss: 0.0647 - val_acc: 0.9867
Epoch 178/1000
2400/2400 [==============================] - 0s 76us/step - loss: 1.7433e-04 - acc: 1.0000 - val_loss: 0.0639 - val_acc: 0.9872
Epoch 179/1000
2400/2400 [==============================] - 0s 60us/step - loss: 1.9454e-04 - acc: 1.0000 - val_loss: 0.0682 - val_acc: 0.9883
Epoch 180/1000
2400/2400 [==============================] - 0s 63us/step - loss: 7.7515e-05 - acc: 1.0000 - val_loss: 0.0683 - val_acc: 0.9878
Epoch 181/1000
2400/2400 [==============================] - 0s 76us/step - loss: 1.7485e-04 - acc: 1.0000 - val_loss: 0.0714 - val_acc: 0.9872
Epoch 182/1000
2400/2400 [==============================] - 0s 70us/step - loss: 1.2206e-04 - acc: 1.0000 - val_loss: 0.0665 - val_acc: 0.9878
Epoch 183/1000

2400/2400 [==============================] - 0s 60us/step - loss: 1.3771e-04 - acc: 1.0000 - val_loss: 0.0700 - val_acc: 0.9856
Epoch 234/1000
2400/2400 [==============================] - 0s 59us/step - loss: 2.3716e-04 - acc: 1.0000 - val_loss: 0.0647 - val_acc: 0.9878
Epoch 235/1000
2400/2400 [==============================] - 0s 62us/step - loss: 6.3363e-05 - acc: 1.0000 - val_loss: 0.0648 - val_acc: 0.9872
Epoch 236/1000
2400/2400 [==============================] - 0s 60us/step - loss: 6.2003e-05 - acc: 1.0000 - val_loss: 0.0659 - val_acc: 0.9867
Epoch 237/1000
2400/2400 [==============================] - 0s 63us/step - loss: 9.6290e-05 - acc: 1.0000 - val_loss: 0.0687 - val_acc: 0.9878
Epoch 238/1000
2400/2400 [==============================] - 0s 66us/step - loss: 4.9289e-05 - acc: 1.0000 - val_loss: 0.0696 - val_acc: 0.9878
Epoch 239/1000
2400/2400 [==============================] - 0s 69us/step - loss: 4.9394e-05 - acc: 1.0000 - val_loss: 0.0724 - val_acc: 0.9878
Epoch 240/1000

...

2400/2400 [==============================] - 0s 64us/step - loss: 4.3220e-05 - acc: 1.0000 - val_loss: 0.1032 - val_acc: 0.9861
Epoch 747/1000
2400/2400 [==============================] - 0s 69us/step - loss: 3.6650e-05 - acc: 1.0000 - val_loss: 0.0972 - val_acc: 0.9867
Epoch 748/1000
2400/2400 [==============================] - 0s 91us/step - loss: 2.0262e-05 - acc: 1.0000 - val_loss: 0.0955 - val_acc: 0.9861
Epoch 749/1000
2400/2400 [==============================] - 0s 77us/step - loss: 2.8587e-05 - acc: 1.0000 - val_loss: 0.0917 - val_acc: 0.9878
Epoch 750/1000
2400/2400 [==============================] - 0s 80us/step - loss: 2.8074e-05 - acc: 1.0000 - val_loss: 0.0879 - val_acc: 0.9872
Epoch 751/1000
2400/2400 [==============================] - 0s 78us/step - loss: 1.0244e-05 - acc: 1.0000 - val_loss: 0.0873 - val_acc: 0.9883
Epoch 752/1000
2400/2400 [==============================] - 0s 86us/step - loss: 1.3685e-05 - acc: 1.0000 - val_loss: 0.0866 - val_acc: 0.9883
Epoch 753/1000

2400/2400 [==============================] - 0s 67us/step - loss: 7.0915e-06 - acc: 1.0000 - val_loss: 0.0875 - val_acc: 0.9883
Epoch 804/1000
2400/2400 [==============================] - 0s 67us/step - loss: 9.6440e-06 - acc: 1.0000 - val_loss: 0.0877 - val_acc: 0.9878
Epoch 805/1000
2400/2400 [==============================] - 0s 78us/step - loss: 6.5451e-06 - acc: 1.0000 - val_loss: 0.0881 - val_acc: 0.9878
Epoch 806/1000
2400/2400 [==============================] - 0s 99us/step - loss: 8.8045e-06 - acc: 1.0000 - val_loss: 0.0869 - val_acc: 0.9883
Epoch 807/1000
2400/2400 [==============================] - 0s 76us/step - loss: 6.0425e-06 - acc: 1.0000 - val_loss: 0.0871 - val_acc: 0.9883
Epoch 808/1000
2400/2400 [==============================] - 0s 86us/step - loss: 5.5912e-06 - acc: 1.0000 - val_loss: 0.0874 - val_acc: 0.9883
Epoch 809/1000
2400/2400 [==============================] - 0s 88us/step - loss: 4.8096e-06 - acc: 1.0000 - val_loss: 0.0875 - val_acc: 0.9883
Epoch 810/1000

2400/2400 [==============================] - 0s 65us/step - loss: 4.7214e-06 - acc: 1.0000 - val_loss: 0.0831 - val_acc: 0.9906
Epoch 861/1000
2400/2400 [==============================] - 0s 63us/step - loss: 4.5262e-06 - acc: 1.0000 - val_loss: 0.0838 - val_acc: 0.9906
Epoch 862/1000
2400/2400 [==============================] - 0s 77us/step - loss: 5.0065e-06 - acc: 1.0000 - val_loss: 0.0840 - val_acc: 0.9894
Epoch 863/1000
2400/2400 [==============================] - 0s 73us/step - loss: 9.1875e-06 - acc: 1.0000 - val_loss: 0.0841 - val_acc: 0.9894
Epoch 864/1000
2400/2400 [==============================] - 0s 80us/step - loss: 2.2607e-05 - acc: 1.0000 - val_loss: 0.0850 - val_acc: 0.9906
Epoch 865/1000
2400/2400 [==============================] - 0s 78us/step - loss: 3.6395e-06 - acc: 1.0000 - val_loss: 0.0848 - val_acc: 0.9894
Epoch 866/1000
2400/2400 [==============================] - 0s 78us/step - loss: 1.5964e-05 - acc: 1.0000 - val_loss: 0.0833 - val_acc: 0.9894
Epoch 867/1000

2400/2400 [==============================] - 0s 66us/step - loss: 2.9557e-06 - acc: 1.0000 - val_loss: 0.0848 - val_acc: 0.9894
Epoch 918/1000
2400/2400 [==============================] - 0s 68us/step - loss: 3.5566e-06 - acc: 1.0000 - val_loss: 0.0851 - val_acc: 0.9894
Epoch 919/1000
2400/2400 [==============================] - 0s 70us/step - loss: 3.8199e-05 - acc: 1.0000 - val_loss: 0.0825 - val_acc: 0.9900
Epoch 920/1000
2400/2400 [==============================] - 0s 68us/step - loss: 3.9548e-06 - acc: 1.0000 - val_loss: 0.0836 - val_acc: 0.9900
Epoch 921/1000
2400/2400 [==============================] - 0s 95us/step - loss: 6.6337e-06 - acc: 1.0000 - val_loss: 0.0833 - val_acc: 0.9900
Epoch 922/1000
2400/2400 [==============================] - 0s 80us/step - loss: 4.7029e-06 - acc: 1.0000 - val_loss: 0.0827 - val_acc: 0.9900
Epoch 923/1000
2400/2400 [==============================] - 0s 79us/step - loss: 4.9064e-06 - acc: 1.0000 - val_loss: 0.0839 - val_acc: 0.9906
Epoch 924/1000

2400/2400 [==============================] - 0s 68us/step - loss: 2.2007e-05 - acc: 1.0000 - val_loss: 0.0915 - val_acc: 0.9889
Epoch 975/1000
2400/2400 [==============================] - 0s 79us/step - loss: 1.0885e-04 - acc: 1.0000 - val_loss: 0.1011 - val_acc: 0.9878
Epoch 976/1000
2400/2400 [==============================] - 0s 77us/step - loss: 2.6741e-05 - acc: 1.0000 - val_loss: 0.0957 - val_acc: 0.9872
Epoch 977/1000
2400/2400 [==============================] - 0s 78us/step - loss: 4.5194e-06 - acc: 1.0000 - val_loss: 0.0949 - val_acc: 0.9872
Epoch 978/1000
2400/2400 [==============================] - 0s 79us/step - loss: 4.8983e-06 - acc: 1.0000 - val_loss: 0.0944 - val_acc: 0.9872
Epoch 979/1000
2400/2400 [==============================] - 0s 75us/step - loss: 7.0483e-06 - acc: 1.0000 - val_loss: 0.0935 - val_acc: 0.9872
Epoch 980/1000
2400/2400 [==============================] - 0s 63us/step - loss: 2.4350e-06 - acc: 1.0000 - val_loss: 0.0932 - val_acc: 0.9878
Epoch 981/1000

In [9]:
gap_model = Sequential()
gap_model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
gap_model.add(Activation('relu'))
gap_model.add(MaxPooling2D((2, 2), padding='same'))

gap_model.add(Conv2D(128, (3, 3), padding='same'))
gap_model.add(Activation('relu'))
gap_model.add(MaxPooling2D((2, 2), padding='same'))

gap_model.add(Conv2D(64, (3, 3), padding='same'))
gap_model.add(Activation('relu'))
gap_model.add(MaxPooling2D((2, 2), padding='same'))

gap_model.add(Conv2D(64, (3, 3), padding='same'))
gap_model.add(Activation('relu'))
gap_model.add(MaxPooling2D((2, 2), padding='same'))
gap_model.add(Dropout(0.25))

gap_model.add(GlobalAveragePooling2D())
gap_model.add(Dense(num_classes))
gap_model.add(Activation('softmax'))

#gap_model.summary()

In [10]:
gap_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
gap_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, validation_data=(x_val, y_val))

score = gap_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0], '\tTest accuracy:', score[1])

Train on 2400 samples, validate on 1800 samples
Epoch 1/1000
2400/2400 [==============================] - 0s 154us/step - loss: 1.7947 - acc: 0.1475 - val_loss: 1.7902 - val_acc: 0.1983
Epoch 2/1000
2400/2400 [==============================] - 0s 50us/step - loss: 1.7905 - acc: 0.1767 - val_loss: 1.7895 - val_acc: 0.1744
Epoch 3/1000
2400/2400 [==============================] - 0s 54us/step - loss: 1.7865 - acc: 0.1921 - val_loss: 1.7884 - val_acc: 0.1672
Epoch 4/1000
2400/2400 [==============================] - 0s 61us/step - loss: 1.7833 - acc: 0.2142 - val_loss: 1.7696 - val_acc: 0.1872
Epoch 5/1000
2400/2400 [==============================] - 0s 52us/step - loss: 1.7763 - acc: 0.2267 - val_loss: 1.7767 - val_acc: 0.3017
Epoch 6/1000
2400/2400 [==============================] - 0s 54us/step - loss: 1.7497 - acc: 0.2629 - val_loss: 1.7668 - val_acc: 0.2972
Epoch 7/1000
2400/2400 [==============================] - 0s 56us/step - loss: 1.7946 - acc: 0.2071 - val_loss: 1.7211 - val_acc:

2400/2400 [==============================] - 0s 55us/step - loss: 0.0481 - acc: 0.9871 - val_loss: 0.0598 - val_acc: 0.9833
Epoch 61/1000
2400/2400 [==============================] - 0s 55us/step - loss: 0.0413 - acc: 0.9929 - val_loss: 0.0636 - val_acc: 0.9828
Epoch 62/1000
2400/2400 [==============================] - 0s 55us/step - loss: 0.0327 - acc: 0.9942 - val_loss: 0.0697 - val_acc: 0.9833
Epoch 63/1000
2400/2400 [==============================] - 0s 57us/step - loss: 0.0355 - acc: 0.9938 - val_loss: 0.0605 - val_acc: 0.9844
Epoch 64/1000
2400/2400 [==============================] - 0s 61us/step - loss: 0.0355 - acc: 0.9912 - val_loss: 0.2505 - val_acc: 0.9194
Epoch 65/1000
2400/2400 [==============================] - 0s 61us/step - loss: 0.2671 - acc: 0.9287 - val_loss: 0.0647 - val_acc: 0.9856
Epoch 66/1000
2400/2400 [==============================] - 0s 62us/step - loss: 0.0389 - acc: 0.9929 - val_loss: 0.0548 - val_acc: 0.9867
Epoch 67/1000
2400/2400 [=======================

2400/2400 [==============================] - 0s 61us/step - loss: 0.0050 - acc: 1.0000 - val_loss: 0.0450 - val_acc: 0.9894
Epoch 120/1000
2400/2400 [==============================] - 0s 63us/step - loss: 0.0048 - acc: 0.9996 - val_loss: 0.0460 - val_acc: 0.9889
Epoch 121/1000
2400/2400 [==============================] - 0s 62us/step - loss: 0.0041 - acc: 0.9996 - val_loss: 0.0413 - val_acc: 0.9900
Epoch 122/1000
2400/2400 [==============================] - 0s 68us/step - loss: 0.0048 - acc: 0.9996 - val_loss: 0.0508 - val_acc: 0.9867
Epoch 123/1000
2400/2400 [==============================] - 0s 65us/step - loss: 0.0106 - acc: 0.9967 - val_loss: 0.0495 - val_acc: 0.9878
Epoch 124/1000
2400/2400 [==============================] - 0s 60us/step - loss: 0.0052 - acc: 0.9992 - val_loss: 0.0419 - val_acc: 0.9894
Epoch 125/1000
2400/2400 [==============================] - 0s 60us/step - loss: 0.0031 - acc: 0.9996 - val_loss: 0.0446 - val_acc: 0.9900
Epoch 126/1000
2400/2400 [================

2400/2400 [==============================] - 0s 54us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0462 - val_acc: 0.9911
Epoch 179/1000
2400/2400 [==============================] - 0s 54us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0487 - val_acc: 0.9883
Epoch 180/1000
2400/2400 [==============================] - 0s 56us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0810 - val_acc: 0.9800
Epoch 181/1000
2400/2400 [==============================] - 0s 59us/step - loss: 0.0044 - acc: 0.9996 - val_loss: 0.0483 - val_acc: 0.9906
Epoch 182/1000
2400/2400 [==============================] - 0s 58us/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0512 - val_acc: 0.9906
Epoch 183/1000
2400/2400 [==============================] - 0s 59us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0477 - val_acc: 0.9906
Epoch 184/1000
2400/2400 [==============================] - 0s 60us/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.0552 - val_acc: 0.9889
Epoch 185/1000
2400/2400 [================

2400/2400 [==============================] - 0s 52us/step - loss: 6.5298e-04 - acc: 1.0000 - val_loss: 0.0523 - val_acc: 0.9900
Epoch 237/1000
2400/2400 [==============================] - 0s 54us/step - loss: 3.6692e-04 - acc: 1.0000 - val_loss: 0.0522 - val_acc: 0.9906
Epoch 238/1000
2400/2400 [==============================] - 0s 55us/step - loss: 7.3512e-04 - acc: 1.0000 - val_loss: 0.0544 - val_acc: 0.9900
Epoch 239/1000
2400/2400 [==============================] - 0s 69us/step - loss: 0.2134 - acc: 0.9533 - val_loss: 0.0342 - val_acc: 0.9917
Epoch 240/1000
2400/2400 [==============================] - 0s 70us/step - loss: 0.0046 - acc: 0.9992 - val_loss: 0.0329 - val_acc: 0.9906
Epoch 241/1000
2400/2400 [==============================] - 0s 60us/step - loss: 0.0030 - acc: 0.9996 - val_loss: 0.0315 - val_acc: 0.9917
Epoch 242/1000
2400/2400 [==============================] - 0s 57us/step - loss: 0.0031 - acc: 0.9996 - val_loss: 0.0350 - val_acc: 0.9906
Epoch 243/1000
2400/2400 [====

...

2400/2400 [==============================] - 0s 56us/step - loss: 2.0921e-05 - acc: 1.0000 - val_loss: 0.0667 - val_acc: 0.9900
Epoch 751/1000
2400/2400 [==============================] - 0s 56us/step - loss: 1.4914e-05 - acc: 1.0000 - val_loss: 0.0652 - val_acc: 0.9900
Epoch 752/1000
2400/2400 [==============================] - 0s 57us/step - loss: 2.3157e-05 - acc: 1.0000 - val_loss: 0.0639 - val_acc: 0.9906
Epoch 753/1000
2400/2400 [==============================] - 0s 55us/step - loss: 2.3723e-05 - acc: 1.0000 - val_loss: 0.0630 - val_acc: 0.9906
Epoch 754/1000
2400/2400 [==============================] - 0s 57us/step - loss: 0.0015 - acc: 0.9992 - val_loss: 0.0541 - val_acc: 0.9911
Epoch 755/1000
2400/2400 [==============================] - 0s 77us/step - loss: 4.4309e-05 - acc: 1.0000 - val_loss: 0.0540 - val_acc: 0.9917
Epoch 756/1000
2400/2400 [==============================] - 0s 78us/step - loss: 8.7584e-05 - acc: 1.0000 - val_loss: 0.0570 - val_acc: 0.9911
Epoch 757/1000
240

2400/2400 [==============================] - 0s 56us/step - loss: 1.9917e-05 - acc: 1.0000 - val_loss: 0.0621 - val_acc: 0.9906
Epoch 808/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.7932e-05 - acc: 1.0000 - val_loss: 0.0615 - val_acc: 0.9900
Epoch 809/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.2404e-05 - acc: 1.0000 - val_loss: 0.0619 - val_acc: 0.9906
Epoch 810/1000
2400/2400 [==============================] - 0s 60us/step - loss: 2.3764e-05 - acc: 1.0000 - val_loss: 0.0624 - val_acc: 0.9911
Epoch 811/1000
2400/2400 [==============================] - 0s 59us/step - loss: 1.0999e-05 - acc: 1.0000 - val_loss: 0.0628 - val_acc: 0.9906
Epoch 812/1000
2400/2400 [==============================] - 0s 59us/step - loss: 1.6183e-05 - acc: 1.0000 - val_loss: 0.0619 - val_acc: 0.9906
Epoch 813/1000
2400/2400 [==============================] - 0s 58us/step - loss: 4.7429e-05 - acc: 1.0000 - val_loss: 0.0683 - val_acc: 0.9889
Epoch 814/1000

2400/2400 [==============================] - 0s 55us/step - loss: 1.6377e-05 - acc: 1.0000 - val_loss: 0.0570 - val_acc: 0.9911
Epoch 865/1000
2400/2400 [==============================] - 0s 56us/step - loss: 2.6718e-05 - acc: 1.0000 - val_loss: 0.0557 - val_acc: 0.9911
Epoch 866/1000
2400/2400 [==============================] - 0s 61us/step - loss: 2.7120e-05 - acc: 1.0000 - val_loss: 0.0563 - val_acc: 0.9906
Epoch 867/1000
2400/2400 [==============================] - 0s 63us/step - loss: 1.6792e-05 - acc: 1.0000 - val_loss: 0.0557 - val_acc: 0.9906
Epoch 868/1000
2400/2400 [==============================] - 0s 68us/step - loss: 1.2125e-05 - acc: 1.0000 - val_loss: 0.0558 - val_acc: 0.9906
Epoch 869/1000
2400/2400 [==============================] - 0s 68us/step - loss: 4.5505e-04 - acc: 0.9996 - val_loss: 0.0658 - val_acc: 0.9894
Epoch 870/1000
2400/2400 [==============================] - 0s 69us/step - loss: 1.4558e-04 - acc: 1.0000 - val_loss: 0.0737 - val_acc: 0.9906
Epoch 871/1000

2400/2400 [==============================] - 0s 56us/step - loss: 1.1071e-05 - acc: 1.0000 - val_loss: 0.0600 - val_acc: 0.9911
Epoch 922/1000
2400/2400 [==============================] - 0s 58us/step - loss: 2.1491e-05 - acc: 1.0000 - val_loss: 0.0590 - val_acc: 0.9911
Epoch 923/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.2782e-05 - acc: 1.0000 - val_loss: 0.0594 - val_acc: 0.9911
Epoch 924/1000
2400/2400 [==============================] - 0s 58us/step - loss: 8.0005e-06 - acc: 1.0000 - val_loss: 0.0597 - val_acc: 0.9911
Epoch 925/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.4707e-05 - acc: 1.0000 - val_loss: 0.0596 - val_acc: 0.9911
Epoch 926/1000
2400/2400 [==============================] - 0s 60us/step - loss: 1.7944e-05 - acc: 1.0000 - val_loss: 0.0618 - val_acc: 0.9906
Epoch 927/1000
2400/2400 [==============================] - 0s 59us/step - loss: 1.8390e-05 - acc: 1.0000 - val_loss: 0.0607 - val_acc: 0.9911
Epoch 928/1000

2400/2400 [==============================] - 0s 54us/step - loss: 4.2825e-05 - acc: 1.0000 - val_loss: 0.0565 - val_acc: 0.9906
Epoch 979/1000
2400/2400 [==============================] - 0s 54us/step - loss: 1.5086e-05 - acc: 1.0000 - val_loss: 0.0554 - val_acc: 0.9917
Epoch 980/1000
2400/2400 [==============================] - 0s 58us/step - loss: 3.8638e-05 - acc: 1.0000 - val_loss: 0.0530 - val_acc: 0.9928
Epoch 981/1000
2400/2400 [==============================] - 0s 61us/step - loss: 1.4846e-05 - acc: 1.0000 - val_loss: 0.0530 - val_acc: 0.9928
Epoch 982/1000
2400/2400 [==============================] - 0s 61us/step - loss: 1.3348e-05 - acc: 1.0000 - val_loss: 0.0528 - val_acc: 0.9928
Epoch 983/1000
2400/2400 [==============================] - 0s 61us/step - loss: 8.7713e-06 - acc: 1.0000 - val_loss: 0.0529 - val_acc: 0.9928
Epoch 984/1000
2400/2400 [==============================] - 0s 61us/step - loss: 1.2515e-05 - acc: 1.0000 - val_loss: 0.0535 - val_acc: 0.9922
Epoch 985/1000

In [12]:
cnnAuto_model = Sequential()
cnnAuto_model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(MaxPooling2D((2, 2), padding='same'))

cnnAuto_model.add(Conv2D(128, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(MaxPooling2D((2, 2), padding='same'))

cnnAuto_model.add(Conv2D(64, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(MaxPooling2D((2, 2), padding='same'))

cnnAuto_model.add(Conv2D(64, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(MaxPooling2D((2, 2), padding='same'))

cnnAuto_model.add(Flatten())
cnnAuto_model.add(Dense(128, activation='relu'))
cnnAuto_model.add(Dropout(0.25))

cnnAuto_model.add(Dense(64, activation = 'relu'))

#cnnAuto_model.add(Dropout(0.25))
cnnAuto_model.add(Dense(num_classes))
cnnAuto_model.add(Activation('softmax'))
#cnnAuto_model.summary()

In [13]:
cnnAuto_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [14]:
cnnAuto_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, validation_data=(x_val, y_val))

score = cnnAuto_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0], 'Test accuracy:', score[1])

Train on 2400 samples, validate on 1800 samples
Epoch 1/1000
2400/2400 [==============================] - 0s 190us/step - loss: 1.7927 - acc: 0.1687 - val_loss: 1.7913 - val_acc: 0.1906
Epoch 2/1000
2400/2400 [==============================] - 0s 54us/step - loss: 1.7912 - acc: 0.1783 - val_loss: 1.7897 - val_acc: 0.2506
Epoch 3/1000
2400/2400 [==============================] - 0s 69us/step - loss: 1.7896 - acc: 0.1737 - val_loss: 1.7940 - val_acc: 0.1667
Epoch 4/1000
2400/2400 [==============================] - 0s 71us/step - loss: 1.7822 - acc: 0.2129 - val_loss: 1.7569 - val_acc: 0.1956
Epoch 5/1000
2400/2400 [==============================] - 0s 80us/step - loss: 1.7934 - acc: 0.1817 - val_loss: 1.7914 - val_acc: 0.1667
Epoch 6/1000
2400/2400 [==============================] - 0s 67us/step - loss: 1.7902 - acc: 0.1721 - val_loss: 1.7847 - val_acc: 0.2678
Epoch 7/1000
2400/2400 [==============================] - 0s 61us/step - loss: 1.7701 - acc: 0.2283 - val_loss: 1.8711 - val_acc:

2400/2400 [==============================] - 0s 70us/step - loss: 0.0078 - acc: 0.9992 - val_loss: 0.0585 - val_acc: 0.9856
Epoch 61/1000
2400/2400 [==============================] - 0s 69us/step - loss: 0.0087 - acc: 0.9975 - val_loss: 0.0600 - val_acc: 0.9844
Epoch 62/1000
2400/2400 [==============================] - 0s 93us/step - loss: 0.0772 - acc: 0.9762 - val_loss: 0.0546 - val_acc: 0.9844
Epoch 63/1000
2400/2400 [==============================] - 0s 89us/step - loss: 0.0097 - acc: 0.9992 - val_loss: 0.0510 - val_acc: 0.9850
Epoch 64/1000
2400/2400 [==============================] - 0s 79us/step - loss: 0.0053 - acc: 0.9996 - val_loss: 0.0515 - val_acc: 0.9828
Epoch 65/1000
2400/2400 [==============================] - 0s 78us/step - loss: 0.0053 - acc: 0.9992 - val_loss: 0.0543 - val_acc: 0.9839
Epoch 66/1000
2400/2400 [==============================] - 0s 79us/step - loss: 0.0043 - acc: 1.0000 - val_loss: 0.0577 - val_acc: 0.9861
Epoch 67/1000
2400/2400 [=======================

Epoch 119/1000
2400/2400 [==============================] - 0s 73us/step - loss: 4.6351e-04 - acc: 1.0000 - val_loss: 0.0593 - val_acc: 0.9861
Epoch 120/1000
2400/2400 [==============================] - 0s 75us/step - loss: 4.4449e-04 - acc: 1.0000 - val_loss: 0.0645 - val_acc: 0.9878
Epoch 121/1000
2400/2400 [==============================] - 0s 82us/step - loss: 4.4761e-04 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 0.9861
Epoch 122/1000
2400/2400 [==============================] - 0s 82us/step - loss: 0.0012 - acc: 0.9996 - val_loss: 0.0789 - val_acc: 0.9878
Epoch 123/1000
2400/2400 [==============================] - 0s 84us/step - loss: 0.0025 - acc: 0.9996 - val_loss: 0.0690 - val_acc: 0.9867
Epoch 124/1000
2400/2400 [==============================] - 0s 82us/step - loss: 6.4351e-04 - acc: 1.0000 - val_loss: 0.0637 - val_acc: 0.9861
Epoch 125/1000
2400/2400 [==============================] - 0s 84us/step - loss: 2.4020e-04 - acc: 1.0000 - val_loss: 0.0639 - val_acc: 0.9878
Epoch 1

2400/2400 [==============================] - 0s 69us/step - loss: 1.1741e-04 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9872
Epoch 177/1000
2400/2400 [==============================] - 0s 74us/step - loss: 1.3388e-04 - acc: 1.0000 - val_loss: 0.0660 - val_acc: 0.9872
Epoch 178/1000
2400/2400 [==============================] - 0s 79us/step - loss: 1.5478e-04 - acc: 1.0000 - val_loss: 0.0746 - val_acc: 0.9872
Epoch 179/1000
2400/2400 [==============================] - 0s 64us/step - loss: 6.7867e-04 - acc: 1.0000 - val_loss: 0.0590 - val_acc: 0.9889
Epoch 180/1000
2400/2400 [==============================] - 0s 56us/step - loss: 1.3863e-04 - acc: 1.0000 - val_loss: 0.0592 - val_acc: 0.9889
Epoch 181/1000
2400/2400 [==============================] - 0s 69us/step - loss: 9.6706e-05 - acc: 1.0000 - val_loss: 0.0662 - val_acc: 0.9878
Epoch 182/1000
2400/2400 [==============================] - 0s 70us/step - loss: 1.0195e-04 - acc: 1.0000 - val_loss: 0.0625 - val_acc: 0.9883
Epoch 183/1000

2400/2400 [==============================] - 0s 58us/step - loss: 9.2295e-05 - acc: 1.0000 - val_loss: 0.0507 - val_acc: 0.9911
Epoch 234/1000
2400/2400 [==============================] - 0s 70us/step - loss: 1.3315e-04 - acc: 1.0000 - val_loss: 0.0537 - val_acc: 0.9906
Epoch 235/1000
2400/2400 [==============================] - 0s 61us/step - loss: 6.7301e-05 - acc: 1.0000 - val_loss: 0.0541 - val_acc: 0.9906
Epoch 236/1000
2400/2400 [==============================] - 0s 66us/step - loss: 4.8051e-05 - acc: 1.0000 - val_loss: 0.0547 - val_acc: 0.9906
Epoch 237/1000
2400/2400 [==============================] - 0s 81us/step - loss: 8.3659e-05 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 0.9906
Epoch 238/1000
2400/2400 [==============================] - 0s 98us/step - loss: 5.3453e-05 - acc: 1.0000 - val_loss: 0.0577 - val_acc: 0.9900
Epoch 239/1000
2400/2400 [==============================] - 0s 83us/step - loss: 6.0706e-05 - acc: 1.0000 - val_loss: 0.0560 - val_acc: 0.9894
Epoch 240/1000

...

2400/2400 [==============================] - 0s 64us/step - loss: 9.1413e-06 - acc: 1.0000 - val_loss: 0.0733 - val_acc: 0.9906
Epoch 747/1000
2400/2400 [==============================] - 0s 70us/step - loss: 1.0324e-05 - acc: 1.0000 - val_loss: 0.0724 - val_acc: 0.9900
Epoch 748/1000
2400/2400 [==============================] - 0s 86us/step - loss: 1.3491e-05 - acc: 1.0000 - val_loss: 0.0751 - val_acc: 0.9900
Epoch 749/1000
2400/2400 [==============================] - 0s 84us/step - loss: 9.6578e-06 - acc: 1.0000 - val_loss: 0.0767 - val_acc: 0.9900
Epoch 750/1000
2400/2400 [==============================] - 0s 75us/step - loss: 4.7622e-06 - acc: 1.0000 - val_loss: 0.0764 - val_acc: 0.9900
Epoch 751/1000
2400/2400 [==============================] - 0s 78us/step - loss: 3.2700e-06 - acc: 1.0000 - val_loss: 0.0763 - val_acc: 0.9900
Epoch 752/1000
2400/2400 [==============================] - 0s 80us/step - loss: 1.3871e-05 - acc: 1.0000 - val_loss: 0.0760 - val_acc: 0.9894
Epoch 753/1000

2400/2400 [==============================] - 0s 73us/step - loss: 3.9303e-06 - acc: 1.0000 - val_loss: 0.0767 - val_acc: 0.9889
Epoch 804/1000
2400/2400 [==============================] - 0s 86us/step - loss: 1.9148e-04 - acc: 1.0000 - val_loss: 0.1210 - val_acc: 0.9811
Epoch 805/1000
2400/2400 [==============================] - 0s 99us/step - loss: 0.0019 - acc: 0.9992 - val_loss: 0.0840 - val_acc: 0.9872
Epoch 806/1000
2400/2400 [==============================] - 0s 83us/step - loss: 3.3952e-05 - acc: 1.0000 - val_loss: 0.0776 - val_acc: 0.9894
Epoch 807/1000
2400/2400 [==============================] - 0s 67us/step - loss: 1.2849e-05 - acc: 1.0000 - val_loss: 0.0774 - val_acc: 0.9900
Epoch 808/1000
2400/2400 [==============================] - 0s 98us/step - loss: 4.7320e-05 - acc: 1.0000 - val_loss: 0.0748 - val_acc: 0.9894
Epoch 809/1000
2400/2400 [==============================] - 0s 99us/step - loss: 2.4230e-05 - acc: 1.0000 - val_loss: 0.0731 - val_acc: 0.9900
Epoch 810/1000
240

2400/2400 [==============================] - 0s 73us/step - loss: 4.2334e-06 - acc: 1.0000 - val_loss: 0.0728 - val_acc: 0.9900
Epoch 861/1000
2400/2400 [==============================] - 0s 59us/step - loss: 2.5141e-06 - acc: 1.0000 - val_loss: 0.0729 - val_acc: 0.9900
Epoch 862/1000
2400/2400 [==============================] - 0s 63us/step - loss: 1.6362e-05 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9889
Epoch 863/1000
2400/2400 [==============================] - 0s 70us/step - loss: 3.0823e-06 - acc: 1.0000 - val_loss: 0.0720 - val_acc: 0.9894
Epoch 864/1000
2400/2400 [==============================] - 0s 72us/step - loss: 4.1291e-06 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9894
Epoch 865/1000
2400/2400 [==============================] - 0s 70us/step - loss: 2.9344e-06 - acc: 1.0000 - val_loss: 0.0718 - val_acc: 0.9894
Epoch 866/1000
2400/2400 [==============================] - 0s 70us/step - loss: 6.1419e-06 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9889
Epoch 867/1000

2400/2400 [==============================] - 0s 77us/step - loss: 2.3739e-06 - acc: 1.0000 - val_loss: 0.0718 - val_acc: 0.9906
Epoch 918/1000
2400/2400 [==============================] - 0s 87us/step - loss: 2.4412e-05 - acc: 1.0000 - val_loss: 0.0810 - val_acc: 0.9911
Epoch 919/1000
2400/2400 [==============================] - 0s 93us/step - loss: 1.5425e-05 - acc: 1.0000 - val_loss: 0.0736 - val_acc: 0.9900
Epoch 920/1000
2400/2400 [==============================] - 0s 74us/step - loss: 2.9957e-06 - acc: 1.0000 - val_loss: 0.0737 - val_acc: 0.9900
Epoch 921/1000
2400/2400 [==============================] - 0s 70us/step - loss: 1.5909e-06 - acc: 1.0000 - val_loss: 0.0736 - val_acc: 0.9894
Epoch 922/1000
2400/2400 [==============================] - 0s 76us/step - loss: 6.5545e-06 - acc: 1.0000 - val_loss: 0.0725 - val_acc: 0.9906
Epoch 923/1000
2400/2400 [==============================] - 0s 75us/step - loss: 2.4188e-06 - acc: 1.0000 - val_loss: 0.0725 - val_acc: 0.9894
Epoch 924/1000

2400/2400 [==============================] - 0s 65us/step - loss: 3.0610e-06 - acc: 1.0000 - val_loss: 0.0718 - val_acc: 0.9900
Epoch 975/1000
2400/2400 [==============================] - 0s 62us/step - loss: 2.1142e-06 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9900
Epoch 976/1000
2400/2400 [==============================] - 0s 83us/step - loss: 4.9155e-06 - acc: 1.0000 - val_loss: 0.0716 - val_acc: 0.9900
Epoch 977/1000
2400/2400 [==============================] - 0s 89us/step - loss: 2.8624e-06 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9900
Epoch 978/1000
2400/2400 [==============================] - 0s 84us/step - loss: 2.1857e-06 - acc: 1.0000 - val_loss: 0.0718 - val_acc: 0.9900
Epoch 979/1000
2400/2400 [==============================] - 0s 76us/step - loss: 3.4272e-06 - acc: 1.0000 - val_loss: 0.0720 - val_acc: 0.9900
Epoch 980/1000
2400/2400 [==============================] - 0s 74us/step - loss: 5.2979e-06 - acc: 1.0000 - val_loss: 0.0718 - val_acc: 0.9900
Epoch 981/1000

In [15]:
model_json = cnnAuto_model.to_json()
with open("origin_cnn_model.json", "w") as json_file : 
    json_file.write(model_json)

In [16]:
cnnAuto_model.save_weights("origin_cnn_model.h5")
print("saved model to disk")

saved model to disk


In [17]:
for i in range(6):
    cnnAuto_model.pop()
#cnnAuto_model.summary()

In [18]:
cnnAuto_model.add(Conv2D(64, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(UpSampling2D((2, 2)))

cnnAuto_model.add(Conv2D(64, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(UpSampling2D((2, 2)))
cnnAuto_model.add(Dropout(0.25))

cnnAuto_model.add(Conv2D(64, (2, 2)))
cnnAuto_model.add(Conv2D(128, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(UpSampling2D((2, 2)))

cnnAuto_model.add(Conv2D(128, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
cnnAuto_model.add(UpSampling2D((2, 2)))

cnnAuto_model.add(Conv2D(1, (3, 3), padding='same'))
cnnAuto_model.add(Activation('relu'))
#cnnAuto_model.summary()

In [19]:
cnnAuto_model.compile(optimizer='adadelta', loss='binary_crossentropy')
#cnnAuto_model.compile(optimizer='adam', loss='mse')

In [20]:
cnnAuto_model.fit(x_train, x_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1, 
        shuffle = True,
        validation_data=(x_val, x_val))

Train on 2400 samples, validate on 1800 samples
Epoch 1/1000
2400/2400 [==============================] - 1s 395us/step - loss: 2.3652 - val_loss: 0.9028
Epoch 2/1000
2400/2400 [==============================] - 0s 107us/step - loss: 0.9560 - val_loss: 0.8754
Epoch 3/1000
2400/2400 [==============================] - 0s 114us/step - loss: 0.8799 - val_loss: 1.9406
Epoch 4/1000
2400/2400 [==============================] - 0s 119us/step - loss: 1.0837 - val_loss: 1.0454
Epoch 5/1000
2400/2400 [==============================] - 0s 108us/step - loss: 0.8056 - val_loss: 0.6891
Epoch 6/1000
2400/2400 [==============================] - 0s 112us/step - loss: 0.7088 - val_loss: 0.6182
Epoch 7/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.6809 - val_loss: 0.6843
Epoch 8/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.6639 - val_loss: 0.5766
Epoch 9/1000
2400/2400 [==============================] - 0s 117us/step - loss: 0.6636 - val_loss: 0.714

2400/2400 [==============================] - 0s 111us/step - loss: 0.5361 - val_loss: 0.5401
Epoch 78/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.5387 - val_loss: 0.5381
Epoch 79/1000
2400/2400 [==============================] - 0s 107us/step - loss: 0.5387 - val_loss: 0.5371
Epoch 80/1000
2400/2400 [==============================] - 0s 111us/step - loss: 0.5403 - val_loss: 0.5356
Epoch 81/1000
2400/2400 [==============================] - 0s 127us/step - loss: 0.5386 - val_loss: 0.5390
Epoch 82/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5409 - val_loss: 0.5438
Epoch 83/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5402 - val_loss: 0.5368
Epoch 84/1000
2400/2400 [==============================] - 0s 112us/step - loss: 0.5408 - val_loss: 0.5362
Epoch 85/1000
2400/2400 [==============================] - 0s 125us/step - loss: 0.5396 - val_loss: 0.5391
Epoch 86/1000
2400/2400 [==========================

2400/2400 [==============================] - 0s 105us/step - loss: 0.5326 - val_loss: 0.5315
Epoch 154/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.5316 - val_loss: 0.5310
Epoch 155/1000
2400/2400 [==============================] - 0s 126us/step - loss: 0.5346 - val_loss: 0.5333
Epoch 156/1000
2400/2400 [==============================] - 0s 110us/step - loss: 0.5312 - val_loss: 0.5338
Epoch 157/1000
2400/2400 [==============================] - 0s 126us/step - loss: 0.5334 - val_loss: 0.5355
Epoch 158/1000
2400/2400 [==============================] - 0s 105us/step - loss: 0.5327 - val_loss: 0.5268
Epoch 159/1000
2400/2400 [==============================] - 0s 104us/step - loss: 0.5323 - val_loss: 0.5323
Epoch 160/1000
2400/2400 [==============================] - 0s 108us/step - loss: 0.5318 - val_loss: 0.5361
Epoch 161/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.5294 - val_loss: 0.5334
Epoch 162/1000
2400/2400 [=================

Epoch 229/1000
2400/2400 [==============================] - 0s 106us/step - loss: 0.5226 - val_loss: 0.5181
Epoch 230/1000
2400/2400 [==============================] - 0s 130us/step - loss: 0.5216 - val_loss: 0.5191
Epoch 231/1000
2400/2400 [==============================] - 0s 119us/step - loss: 0.5215 - val_loss: 0.5173
Epoch 232/1000
2400/2400 [==============================] - 0s 127us/step - loss: 0.5253 - val_loss: 0.5146
Epoch 233/1000
2400/2400 [==============================] - 0s 116us/step - loss: 0.5192 - val_loss: 0.5152
Epoch 234/1000
2400/2400 [==============================] - 0s 124us/step - loss: 0.5222 - val_loss: 0.5182
Epoch 235/1000
2400/2400 [==============================] - 0s 122us/step - loss: 0.5216 - val_loss: 0.5223
Epoch 236/1000
2400/2400 [==============================] - 0s 110us/step - loss: 0.5213 - val_loss: 0.5189
Epoch 237/1000
2400/2400 [==============================] - 0s 107us/step - loss: 0.5228 - val_loss: 0.5185
Epoch 238/1000
2400/2400 [==

Epoch 305/1000
2400/2400 [==============================] - 0s 107us/step - loss: 0.5167 - val_loss: 0.5097
Epoch 306/1000
2400/2400 [==============================] - 0s 108us/step - loss: 0.5170 - val_loss: 0.5124
Epoch 307/1000
2400/2400 [==============================] - 0s 110us/step - loss: 0.5177 - val_loss: 0.5114
Epoch 308/1000
2400/2400 [==============================] - 0s 119us/step - loss: 0.5187 - val_loss: 0.5100
Epoch 309/1000
2400/2400 [==============================] - 0s 121us/step - loss: 0.5183 - val_loss: 0.5105
Epoch 310/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5174 - val_loss: 0.5101
Epoch 311/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.5175 - val_loss: 0.5089
Epoch 312/1000
2400/2400 [==============================] - 0s 111us/step - loss: 0.5162 - val_loss: 0.5103
Epoch 313/1000
2400/2400 [==============================] - 0s 122us/step - loss: 0.5177 - val_loss: 0.5094
Epoch 314/1000
2400/2400 [==

...

Epoch 685/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.5076 - val_loss: 0.5006
Epoch 686/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.5058 - val_loss: 0.5001
Epoch 687/1000
2400/2400 [==============================] - 0s 134us/step - loss: 0.5064 - val_loss: 0.5003
Epoch 688/1000
2400/2400 [==============================] - 0s 117us/step - loss: 0.5054 - val_loss: 0.4999
Epoch 689/1000
2400/2400 [==============================] - 0s 124us/step - loss: 0.5061 - val_loss: 0.5022
Epoch 690/1000
2400/2400 [==============================] - 0s 121us/step - loss: 0.5059 - val_loss: 0.5060
Epoch 691/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5060 - val_loss: 0.4996
Epoch 692/1000
2400/2400 [==============================] - 0s 133us/step - loss: 0.5067 - val_loss: 0.5129
Epoch 693/1000
2400/2400 [==============================] - 0s 125us/step - loss: 0.5086 - val_loss: 0.5003
Epoch 694/1000
2400/2400 [==

Epoch 761/1000
2400/2400 [==============================] - 0s 114us/step - loss: 0.5051 - val_loss: 0.5058
Epoch 762/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5060 - val_loss: 0.4996
Epoch 763/1000
2400/2400 [==============================] - 0s 116us/step - loss: 0.5042 - val_loss: 0.5000
Epoch 764/1000
2400/2400 [==============================] - 0s 122us/step - loss: 0.5042 - val_loss: 0.4989
Epoch 765/1000
2400/2400 [==============================] - 0s 112us/step - loss: 0.5047 - val_loss: 0.5002
Epoch 766/1000
2400/2400 [==============================] - 0s 114us/step - loss: 0.5062 - val_loss: 0.4992
Epoch 767/1000
2400/2400 [==============================] - 0s 125us/step - loss: 0.5046 - val_loss: 0.4994
Epoch 768/1000
2400/2400 [==============================] - 0s 135us/step - loss: 0.5074 - val_loss: 0.4992
Epoch 769/1000
2400/2400 [==============================] - 0s 112us/step - loss: 0.5042 - val_loss: 0.4988
Epoch 770/1000
2400/2400 [==

Epoch 837/1000
2400/2400 [==============================] - 0s 111us/step - loss: 0.5050 - val_loss: 0.4998
Epoch 838/1000
2400/2400 [==============================] - 0s 114us/step - loss: 0.5049 - val_loss: 0.5039
Epoch 839/1000
2400/2400 [==============================] - 0s 109us/step - loss: 0.5050 - val_loss: 0.4998
Epoch 840/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.5050 - val_loss: 0.4999
Epoch 841/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.5045 - val_loss: 0.4995
Epoch 842/1000
2400/2400 [==============================] - 0s 131us/step - loss: 0.5041 - val_loss: 0.5007
Epoch 843/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.5053 - val_loss: 0.4996
Epoch 844/1000
2400/2400 [==============================] - 0s 127us/step - loss: 0.5050 - val_loss: 0.5031
Epoch 845/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.5065 - val_loss: 0.4999
Epoch 846/1000
2400/2400 [==

Epoch 913/1000
2400/2400 [==============================] - 0s 116us/step - loss: 0.5059 - val_loss: 0.4984
Epoch 914/1000
2400/2400 [==============================] - 0s 119us/step - loss: 0.5041 - val_loss: 0.4982
Epoch 915/1000
2400/2400 [==============================] - 0s 108us/step - loss: 0.5040 - val_loss: 0.4983
Epoch 916/1000
2400/2400 [==============================] - 0s 111us/step - loss: 0.5037 - val_loss: 0.4993
Epoch 917/1000
2400/2400 [==============================] - 0s 116us/step - loss: 0.5042 - val_loss: 0.4997
Epoch 918/1000
2400/2400 [==============================] - 0s 119us/step - loss: 0.5044 - val_loss: 0.5003
Epoch 919/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5053 - val_loss: 0.4980
Epoch 920/1000
2400/2400 [==============================] - 0s 133us/step - loss: 0.5030 - val_loss: 0.4976
Epoch 921/1000
2400/2400 [==============================] - 0s 136us/step - loss: 0.5042 - val_loss: 0.4981
Epoch 922/1000
2400/2400 [==

Epoch 989/1000
2400/2400 [==============================] - 0s 119us/step - loss: 0.5029 - val_loss: 0.4980
Epoch 990/1000
2400/2400 [==============================] - 0s 114us/step - loss: 0.5025 - val_loss: 0.4977
Epoch 991/1000
2400/2400 [==============================] - 0s 132us/step - loss: 0.5025 - val_loss: 0.4982
Epoch 992/1000
2400/2400 [==============================] - 0s 112us/step - loss: 0.5029 - val_loss: 0.4995
Epoch 993/1000
2400/2400 [==============================] - 0s 118us/step - loss: 0.5035 - val_loss: 0.4983
Epoch 994/1000
2400/2400 [==============================] - 0s 106us/step - loss: 0.5023 - val_loss: 0.4977
Epoch 995/1000
2400/2400 [==============================] - 0s 109us/step - loss: 0.5030 - val_loss: 0.4992
Epoch 996/1000
2400/2400 [==============================] - 0s 115us/step - loss: 0.5030 - val_loss: 0.5114
Epoch 997/1000
2400/2400 [==============================] - 0s 113us/step - loss: 0.5064 - val_loss: 0.4988
Epoch 998/1000
2400/2400 [==

In [21]:
pred = cnnAuto_model.predict(x_test)
pred = np.vstack([x_train, pred])

cnnAuto_model.fit(pred, pred,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  shuffle = True,
                  validation_data=(x_val, x_val))

Train on 3600 samples, validate on 1800 samples
Epoch 1/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4992 - val_loss: 0.4982
Epoch 2/1000
3600/3600 [==============================] - 0s 101us/step - loss: 0.5020 - val_loss: 0.5056
Epoch 3/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4998 - val_loss: 0.4996
Epoch 4/1000
3600/3600 [==============================] - 0s 99us/step - loss: 0.5005 - val_loss: 0.5037
Epoch 5/1000
3600/3600 [==============================] - 0s 101us/step - loss: 0.5006 - val_loss: 0.5158
Epoch 6/1000
3600/3600 [==============================] - 0s 105us/step - loss: 0.5029 - val_loss: 0.4997
Epoch 7/1000
3600/3600 [==============================] - 0s 108us/step - loss: 0.4997 - val_loss: 0.4987
Epoch 8/1000
3600/3600 [==============================] - 0s 108us/step - loss: 0.4996 - val_loss: 0.4990
Epoch 9/1000
3600/3600 [==============================] - 0s 99us/step - loss: 0.4992 - val_loss: 0.4980


3600/3600 [==============================] - 0s 102us/step - loss: 0.4993 - val_loss: 0.4969
Epoch 78/1000
3600/3600 [==============================] - 0s 106us/step - loss: 0.4991 - val_loss: 0.4980
Epoch 79/1000
3600/3600 [==============================] - 0s 113us/step - loss: 0.4986 - val_loss: 0.4973
Epoch 80/1000
3600/3600 [==============================] - 0s 103us/step - loss: 0.4993 - val_loss: 0.4995
Epoch 81/1000
3600/3600 [==============================] - 0s 101us/step - loss: 0.4992 - val_loss: 0.5011
Epoch 82/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4995 - val_loss: 0.4986
Epoch 83/1000
3600/3600 [==============================] - 0s 113us/step - loss: 0.4996 - val_loss: 0.4992
Epoch 84/1000
3600/3600 [==============================] - 0s 100us/step - loss: 0.4989 - val_loss: 0.4966
Epoch 85/1000
3600/3600 [==============================] - 0s 100us/step - loss: 0.4988 - val_loss: 0.4966
Epoch 86/1000
3600/3600 [==========================

3600/3600 [==============================] - 0s 104us/step - loss: 0.4986 - val_loss: 0.4971
Epoch 154/1000
3600/3600 [==============================] - 0s 109us/step - loss: 0.4988 - val_loss: 0.4980
Epoch 155/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4979 - val_loss: 0.4975
Epoch 156/1000
3600/3600 [==============================] - 0s 99us/step - loss: 0.4984 - val_loss: 0.4997
Epoch 157/1000
3600/3600 [==============================] - 0s 101us/step - loss: 0.4986 - val_loss: 0.4992
Epoch 158/1000
3600/3600 [==============================] - 0s 104us/step - loss: 0.5032 - val_loss: 0.4969
Epoch 159/1000
3600/3600 [==============================] - 0s 103us/step - loss: 0.4975 - val_loss: 0.4963
Epoch 160/1000
3600/3600 [==============================] - 0s 103us/step - loss: 0.4978 - val_loss: 0.5037
Epoch 161/1000
3600/3600 [==============================] - 0s 109us/step - loss: 0.4990 - val_loss: 0.4968
Epoch 162/1000
3600/3600 [==================

3600/3600 [==============================] - 0s 107us/step - loss: 0.4974 - val_loss: 0.4989
Epoch 230/1000
3600/3600 [==============================] - 0s 105us/step - loss: 0.5006 - val_loss: 0.4970
Epoch 231/1000
3600/3600 [==============================] - 0s 103us/step - loss: 0.4975 - val_loss: 0.4989
Epoch 232/1000
3600/3600 [==============================] - 0s 103us/step - loss: 0.4979 - val_loss: 0.4968
Epoch 233/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4973 - val_loss: 0.4977
Epoch 234/1000
3600/3600 [==============================] - 0s 112us/step - loss: 0.4977 - val_loss: 0.4957
Epoch 235/1000
3600/3600 [==============================] - 0s 104us/step - loss: 0.4992 - val_loss: 0.4977
Epoch 236/1000
3600/3600 [==============================] - 0s 104us/step - loss: 0.4978 - val_loss: 0.4959
Epoch 237/1000
3600/3600 [==============================] - 0s 108us/step - loss: 0.4984 - val_loss: 0.4972
Epoch 238/1000
3600/3600 [=================

3600/3600 [==============================] - 0s 102us/step - loss: 0.4972 - val_loss: 0.4966
Epoch 306/1000
3600/3600 [==============================] - 0s 106us/step - loss: 0.4981 - val_loss: 0.4961
Epoch 307/1000
3600/3600 [==============================] - 0s 109us/step - loss: 0.4974 - val_loss: 0.4960
Epoch 308/1000
3600/3600 [==============================] - 0s 102us/step - loss: 0.4976 - val_loss: 0.4968
Epoch 309/1000
3600/3600 [==============================] - 0s 104us/step - loss: 0.4980 - val_loss: 0.4986
Epoch 310/1000
3600/3600 [==============================] - 0s 106us/step - loss: 0.4995 - val_loss: 0.4964
Epoch 311/1000
3600/3600 [==============================] - 0s 112us/step - loss: 0.4969 - val_loss: 0.4960
Epoch 312/1000
3600/3600 [==============================] - 0s 108us/step - loss: 0.4967 - val_loss: 0.4956
Epoch 313/1000
3600/3600 [==============================] - 0s 108us/step - loss: 0.4970 - val_loss: 0.4972
Epoch 314/1000
3600/3600 [=================

...

3600/3600 [==============================] - 0s 101us/step - loss: 0.4977 - val_loss: 0.4952
Epoch 686/1000
3600/3600 [==============================] - 0s 102us/step - loss: 0.4962 - val_loss: 0.4975
Epoch 687/1000
3600/3600 [==============================] - 0s 98us/step - loss: 0.4969 - val_loss: 0.4957
Epoch 688/1000
3600/3600 [==============================] - 0s 102us/step - loss: 0.4973 - val_loss: 0.4963
Epoch 689/1000
3600/3600 [==============================] - 0s 110us/step - loss: 0.4962 - val_loss: 0.4956
Epoch 690/1000
3600/3600 [==============================] - 0s 98us/step - loss: 0.4986 - val_loss: 0.4958
Epoch 691/1000
3600/3600 [==============================] - 0s 101us/step - loss: 0.4965 - val_loss: 0.4944
Epoch 692/1000
3600/3600 [==============================] - 0s 111us/step - loss: 0.4959 - val_loss: 0.4949
Epoch 693/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4959 - val_loss: 0.4972
Epoch 694/1000
3600/3600 [===================

3600/3600 [==============================] - 0s 107us/step - loss: 0.4960 - val_loss: 0.4960
Epoch 762/1000
3600/3600 [==============================] - 0s 116us/step - loss: 0.4962 - val_loss: 0.4950
Epoch 763/1000
3600/3600 [==============================] - 0s 121us/step - loss: 0.4959 - val_loss: 0.4953
Epoch 764/1000
3600/3600 [==============================] - 0s 128us/step - loss: 0.4964 - val_loss: 0.4975
Epoch 765/1000
3600/3600 [==============================] - 0s 122us/step - loss: 0.4964 - val_loss: 0.4950
Epoch 766/1000
3600/3600 [==============================] - 0s 105us/step - loss: 0.4962 - val_loss: 0.4978
Epoch 767/1000
3600/3600 [==============================] - 0s 100us/step - loss: 0.4966 - val_loss: 0.5063
Epoch 768/1000
3600/3600 [==============================] - 0s 104us/step - loss: 0.4979 - val_loss: 0.4948
Epoch 769/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4960 - val_loss: 0.4959
Epoch 770/1000
3600/3600 [=================

3600/3600 [==============================] - 0s 109us/step - loss: 0.4957 - val_loss: 0.4949
Epoch 838/1000
3600/3600 [==============================] - 0s 106us/step - loss: 0.4956 - val_loss: 0.4957
Epoch 839/1000
3600/3600 [==============================] - 0s 105us/step - loss: 0.4957 - val_loss: 0.4954
Epoch 840/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4960 - val_loss: 0.4947
Epoch 841/1000
3600/3600 [==============================] - 0s 112us/step - loss: 0.4962 - val_loss: 0.4948
Epoch 842/1000
3600/3600 [==============================] - 0s 103us/step - loss: 0.4965 - val_loss: 0.4960
Epoch 843/1000
3600/3600 [==============================] - 0s 102us/step - loss: 0.4966 - val_loss: 0.4951
Epoch 844/1000
3600/3600 [==============================] - 0s 111us/step - loss: 0.4955 - val_loss: 0.4944
Epoch 845/1000
3600/3600 [==============================] - 0s 100us/step - loss: 0.4956 - val_loss: 0.4951
Epoch 846/1000
3600/3600 [=================

3600/3600 [==============================] - 0s 106us/step - loss: 0.4969 - val_loss: 0.4951
Epoch 914/1000
3600/3600 [==============================] - 0s 107us/step - loss: 0.4957 - val_loss: 0.4950
Epoch 915/1000
3600/3600 [==============================] - 0s 113us/step - loss: 0.4955 - val_loss: 0.4948
Epoch 916/1000
3600/3600 [==============================] - 0s 112us/step - loss: 0.4958 - val_loss: 0.4983
Epoch 917/1000
3600/3600 [==============================] - 0s 116us/step - loss: 0.4964 - val_loss: 0.4954
Epoch 918/1000
3600/3600 [==============================] - 0s 110us/step - loss: 0.4956 - val_loss: 0.4959
Epoch 919/1000
3600/3600 [==============================] - 0s 100us/step - loss: 0.4963 - val_loss: 0.4949
Epoch 920/1000
3600/3600 [==============================] - 0s 104us/step - loss: 0.4957 - val_loss: 0.4958
Epoch 921/1000
3600/3600 [==============================] - 0s 121us/step - loss: 0.4965 - val_loss: 0.4962
Epoch 922/1000
3600/3600 [=================

3600/3600 [==============================] - 0s 105us/step - loss: 0.4964 - val_loss: 0.4949
Epoch 990/1000
3600/3600 [==============================] - 0s 113us/step - loss: 0.4959 - val_loss: 0.4948
Epoch 991/1000
3600/3600 [==============================] - 0s 113us/step - loss: 0.4961 - val_loss: 0.4954
Epoch 992/1000
3600/3600 [==============================] - 0s 119us/step - loss: 0.4959 - val_loss: 0.4946
Epoch 993/1000
3600/3600 [==============================] - 0s 110us/step - loss: 0.4958 - val_loss: 0.4964
Epoch 994/1000
3600/3600 [==============================] - 0s 106us/step - loss: 0.4963 - val_loss: 0.4952
Epoch 995/1000
3600/3600 [==============================] - 0s 100us/step - loss: 0.4969 - val_loss: 0.4977
Epoch 996/1000
3600/3600 [==============================] - 0s 101us/step - loss: 0.4970 - val_loss: 0.4956
Epoch 997/1000
3600/3600 [==============================] - 0s 102us/step - loss: 0.4961 - val_loss: 0.4956
Epoch 998/1000
3600/3600 [=================

In [22]:
model_json = cnnAuto_model.to_json()
with open("cnnAuto_model.json", "w") as json_file : 
    json_file.write(model_json)
cnnAuto_model.save_weights("cnnAuto_model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
json_file = open("cnnAuto_model.json", "r") 
cnnAuto_gap_model = json_file.read() 
json_file.close() 
cnnAuto_gap_model = model_from_json(cnnAuto_gap_model)
cnnAuto_gap_model.load_weights("cnnAuto_model.h5") 

json_file1 = open("cnnAuto_model.json", "r") 
cnnAuto_dense_model = json_file1.read()
json_file1.close() 
cnnAuto_dense_model = model_from_json(cnnAuto_dense_model)
cnnAuto_dense_model.load_weights("cnnAuto_model.h5")
print("Loaded model from disk")

for i in range(16):
    cnnAuto_gap_model.pop()
    cnnAuto_dense_model.pop()
    
cnnAuto_gap_model.add(GlobalAveragePooling2D())
cnnAuto_gap_model.add(Dense(num_classes))

cnnAuto_dense_model.add(Flatten())
cnnAuto_dense_model.add(Dense(128, activation='relu'))
cnnAuto_dense_model.add(Dropout(0.25))
cnnAuto_dense_model.add(Dense(64, activation = 'relu'))
cnnAuto_dense_model.add(Dense(num_classes))
cnnAuto_dense_model.add(Activation('softmax'))

Loaded model from disk


In [ ]:
i = 0
for l1, l2 in zip(cnnAuto_gap_model.layers, cnnAuto_dense_model.layers):
    l1.trainable = False
    l2.trainable = False
    i= i+1
    if i == 12:
        break

In [ ]:
cnnAuto_gap_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

cnnAuto_dense_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
cnnAuto_gap_model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1, 
        shuffle = True,
        validation_data=(x_val, y_val))

Train on 2400 samples, validate on 1800 samples
Epoch 1/1000
2400/2400 [==============================] - 0s 141us/step - loss: 7.0247 - acc: 0.2942 - val_loss: 6.0716 - val_acc: 0.2728
Epoch 2/1000
2400/2400 [==============================] - 0s 31us/step - loss: 5.7571 - acc: 0.2546 - val_loss: 5.2365 - val_acc: 0.1856
Epoch 3/1000
2400/2400 [==============================] - 0s 36us/step - loss: 5.1753 - acc: 0.1913 - val_loss: 5.1379 - val_acc: 0.1700
Epoch 4/1000
2400/2400 [==============================] - 0s 38us/step - loss: 4.6037 - acc: 0.1596 - val_loss: 4.7261 - val_acc: 0.1583
Epoch 5/1000
2400/2400 [==============================] - 0s 35us/step - loss: 4.8703 - acc: 0.1588 - val_loss: 4.5830 - val_acc: 0.1511
Epoch 6/1000
2400/2400 [==============================] - 0s 34us/step - loss: 4.6385 - acc: 0.1554 - val_loss: 4.8486 - val_acc: 0.1517
Epoch 7/1000
2400/2400 [==============================] - 0s 37us/step - loss: 4.6168 - acc: 0.1558 - val_loss: 4.5416 - val_acc:

2400/2400 [==============================] - 0s 38us/step - loss: 6.8007 - acc: 0.1637 - val_loss: 7.1277 - val_acc: 0.1622
Epoch 61/1000
2400/2400 [==============================] - 0s 36us/step - loss: 6.8303 - acc: 0.1654 - val_loss: 7.1899 - val_acc: 0.1622
Epoch 62/1000
2400/2400 [==============================] - 0s 43us/step - loss: 6.9165 - acc: 0.1642 - val_loss: 7.2803 - val_acc: 0.1628
Epoch 63/1000
2400/2400 [==============================] - 0s 44us/step - loss: 6.9034 - acc: 0.1654 - val_loss: 7.2386 - val_acc: 0.1628
Epoch 64/1000
2400/2400 [==============================] - 0s 42us/step - loss: 6.9046 - acc: 0.1650 - val_loss: 7.0534 - val_acc: 0.1628
Epoch 65/1000
2400/2400 [==============================] - 0s 42us/step - loss: 7.0100 - acc: 0.1654 - val_loss: 7.2114 - val_acc: 0.1628
Epoch 66/1000
2400/2400 [==============================] - 0s 44us/step - loss: 7.0103 - acc: 0.1654 - val_loss: 7.1929 - val_acc: 0.1633
Epoch 67/1000
2400/2400 [=======================

2400/2400 [==============================] - 0s 34us/step - loss: 8.4191 - acc: 0.1638 - val_loss: 8.9043 - val_acc: 0.1617
Epoch 120/1000
2400/2400 [==============================] - 0s 34us/step - loss: 8.4470 - acc: 0.1629 - val_loss: 9.5701 - val_acc: 0.1600
Epoch 121/1000
2400/2400 [==============================] - 0s 45us/step - loss: 8.9922 - acc: 0.1633 - val_loss: 9.3761 - val_acc: 0.1622
Epoch 122/1000
2400/2400 [==============================] - 0s 46us/step - loss: 8.8243 - acc: 0.1650 - val_loss: 8.8054 - val_acc: 0.1622
Epoch 123/1000
2400/2400 [==============================] - 0s 44us/step - loss: 8.5321 - acc: 0.1650 - val_loss: 9.1446 - val_acc: 0.1622
Epoch 124/1000
2400/2400 [==============================] - 0s 44us/step - loss: 8.5650 - acc: 0.1650 - val_loss: 8.6877 - val_acc: 0.1622
Epoch 125/1000
2400/2400 [==============================] - 0s 45us/step - loss: 8.5103 - acc: 0.1646 - val_loss: 8.5058 - val_acc: 0.1628
Epoch 126/1000
2400/2400 [================

2400/2400 [==============================] - 0s 33us/step - loss: 8.3925 - acc: 0.1646 - val_loss: 8.5896 - val_acc: 0.1633
Epoch 179/1000
2400/2400 [==============================] - 0s 34us/step - loss: 8.2304 - acc: 0.1654 - val_loss: 8.1410 - val_acc: 0.1633
Epoch 180/1000
2400/2400 [==============================] - 0s 36us/step - loss: 7.9420 - acc: 0.1654 - val_loss: 8.6286 - val_acc: 0.1633
Epoch 181/1000
2400/2400 [==============================] - 0s 38us/step - loss: 7.9221 - acc: 0.1654 - val_loss: 8.1541 - val_acc: 0.1639
Epoch 182/1000
2400/2400 [==============================] - 0s 36us/step - loss: 8.2817 - acc: 0.1663 - val_loss: 8.2281 - val_acc: 0.1639
Epoch 183/1000
2400/2400 [==============================] - 0s 40us/step - loss: 8.2797 - acc: 0.1654 - val_loss: 9.0004 - val_acc: 0.1639
Epoch 184/1000
2400/2400 [==============================] - 0s 39us/step - loss: 8.1285 - acc: 0.1654 - val_loss: 8.3100 - val_acc: 0.1639
Epoch 185/1000
2400/2400 [================

2400/2400 [==============================] - 0s 40us/step - loss: 8.5225 - acc: 0.1663 - val_loss: 9.0999 - val_acc: 0.1639
Epoch 238/1000
2400/2400 [==============================] - 0s 36us/step - loss: 8.5380 - acc: 0.1663 - val_loss: 8.1742 - val_acc: 0.1639
Epoch 239/1000
2400/2400 [==============================] - 0s 50us/step - loss: 8.3355 - acc: 0.1662 - val_loss: 8.1046 - val_acc: 0.1639
Epoch 240/1000
2400/2400 [==============================] - 0s 49us/step - loss: 7.8804 - acc: 0.1662 - val_loss: 8.0618 - val_acc: 0.1639
Epoch 241/1000
2400/2400 [==============================] - 0s 42us/step - loss: 9.2185 - acc: 0.1663 - val_loss: 8.8756 - val_acc: 0.1639
Epoch 242/1000
2400/2400 [==============================] - 0s 42us/step - loss: 9.0708 - acc: 0.1662 - val_loss: 9.3491 - val_acc: 0.1639
Epoch 243/1000
2400/2400 [==============================] - 0s 43us/step - loss: 8.8377 - acc: 0.1662 - val_loss: 8.1038 - val_acc: 0.1639
Epoch 244/1000
2400/2400 [================

...

Epoch 768/1000
2400/2400 [==============================] - 0s 36us/step - loss: 9.7238 - acc: 0.2746 - val_loss: 7.0228 - val_acc: 0.2583
Epoch 769/1000
2400/2400 [==============================] - 0s 34us/step - loss: 8.3691 - acc: 0.2767 - val_loss: 7.8474 - val_acc: 0.2606
Epoch 770/1000
2400/2400 [==============================] - 0s 41us/step - loss: 7.6937 - acc: 0.2871 - val_loss: 10.0641 - val_acc: 0.2772
Epoch 771/1000
2400/2400 [==============================] - 0s 41us/step - loss: 10.5644 - acc: 0.2975 - val_loss: 10.1672 - val_acc: 0.2761
Epoch 772/1000
2400/2400 [==============================] - 0s 40us/step - loss: 9.6401 - acc: 0.3008 - val_loss: 8.7991 - val_acc: 0.2700
Epoch 773/1000
2400/2400 [==============================] - 0s 36us/step - loss: 9.8056 - acc: 0.2971 - val_loss: 9.6115 - val_acc: 0.2689
Epoch 774/1000
2400/2400 [==============================] - 0s 39us/step - loss: 9.4102 - acc: 0.2979 - val_loss: 10.1107 - val_acc: 0.2706
Epoch 775/1000
2400/240

2400/2400 [==============================] - 0s 35us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 826/1000
2400/2400 [==============================] - 0s 34us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 827/1000
2400/2400 [==============================] - 0s 35us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 828/1000
2400/2400 [==============================] - 0s 36us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 829/1000
2400/2400 [==============================] - 0s 38us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 830/1000
2400/2400 [==============================] - 0s 37us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 831/1000
2400/2400 [==============================] - 0s 36us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - va

Epoch 881/1000
2400/2400 [==============================] - 0s 37us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 882/1000
2400/2400 [==============================] - 0s 48us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 883/1000
2400/2400 [==============================] - 0s 49us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 884/1000
2400/2400 [==============================] - 0s 48us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 885/1000
2400/2400 [==============================] - 0s 47us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 886/1000
2400/2400 [==============================] - 0s 47us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 887/1000
2400/2400 [==============================] - 0s 50us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 

Epoch 937/1000
2400/2400 [==============================] - 0s 35us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 938/1000
2400/2400 [==============================] - 0s 41us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 939/1000
2400/2400 [==============================] - 0s 41us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 940/1000
2400/2400 [==============================] - 0s 41us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 941/1000
2400/2400 [==============================] - 0s 38us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 942/1000
2400/2400 [==============================] - 0s 42us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 943/1000
2400/2400 [==============================] - 0s 42us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 

Epoch 993/1000
2400/2400 [==============================] - 0s 30us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 994/1000
2400/2400 [==============================] - 0s 36us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 995/1000
2400/2400 [==============================] - 0s 37us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 996/1000
2400/2400 [==============================] - 0s 42us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 997/1000
2400/2400 [==============================] - 0s 41us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 998/1000
2400/2400 [==============================] - 0s 42us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 1.1921e-07 - val_acc: 0.1667
Epoch 999/1000
2400/2400 [==============================] - 0s 41us/step - loss: 1.1921e-07 - acc: 0.1667 - val_loss: 

In [ ]:
cnnAuto_dense_model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1, 
        shuffle = True,
        validation_data=(x_val, y_val))

Train on 2400 samples, validate on 1800 samples
Epoch 1/1000
2400/2400 [==============================] - 0s 171us/step - loss: 1.7028 - acc: 0.3342 - val_loss: 1.5216 - val_acc: 0.6894
Epoch 2/1000
2400/2400 [==============================] - 0s 37us/step - loss: 1.3551 - acc: 0.7225 - val_loss: 1.0489 - val_acc: 0.8061
Epoch 3/1000
2400/2400 [==============================] - 0s 53us/step - loss: 0.9148 - acc: 0.8221 - val_loss: 0.6787 - val_acc: 0.8811
Epoch 4/1000
2400/2400 [==============================] - 0s 68us/step - loss: 0.6105 - acc: 0.8817 - val_loss: 0.4454 - val_acc: 0.9111
Epoch 5/1000
2400/2400 [==============================] - 0s 46us/step - loss: 0.4377 - acc: 0.9088 - val_loss: 0.3588 - val_acc: 0.9061
Epoch 6/1000
2400/2400 [==============================] - 0s 53us/step - loss: 0.3499 - acc: 0.9183 - val_loss: 0.2945 - val_acc: 0.9183
Epoch 7/1000
2400/2400 [==============================] - 0s 55us/step - loss: 0.2889 - acc: 0.9317 - val_loss: 0.2418 - val_acc:

2400/2400 [==============================] - 0s 50us/step - loss: 0.0202 - acc: 0.9967 - val_loss: 0.0820 - val_acc: 0.9778
Epoch 61/1000
2400/2400 [==============================] - 0s 53us/step - loss: 0.0193 - acc: 0.9975 - val_loss: 0.0837 - val_acc: 0.9789
Epoch 62/1000
2400/2400 [==============================] - 0s 80us/step - loss: 0.0209 - acc: 0.9942 - val_loss: 0.0842 - val_acc: 0.9783
Epoch 63/1000
2400/2400 [==============================] - 0s 73us/step - loss: 0.0162 - acc: 0.9979 - val_loss: 0.0830 - val_acc: 0.9789
Epoch 64/1000
2400/2400 [==============================] - 0s 56us/step - loss: 0.0150 - acc: 0.9983 - val_loss: 0.0890 - val_acc: 0.9778
Epoch 65/1000
2400/2400 [==============================] - 0s 55us/step - loss: 0.0194 - acc: 0.9954 - val_loss: 0.0801 - val_acc: 0.9778
Epoch 66/1000
2400/2400 [==============================] - 0s 50us/step - loss: 0.0155 - acc: 0.9988 - val_loss: 0.0829 - val_acc: 0.9767
Epoch 67/1000
2400/2400 [=======================

2400/2400 [==============================] - 0s 35us/step - loss: 0.0054 - acc: 0.9992 - val_loss: 0.0869 - val_acc: 0.9783
Epoch 120/1000
2400/2400 [==============================] - 0s 51us/step - loss: 0.0070 - acc: 0.9992 - val_loss: 0.0829 - val_acc: 0.9789
Epoch 121/1000
2400/2400 [==============================] - 0s 51us/step - loss: 0.0060 - acc: 0.9988 - val_loss: 0.0850 - val_acc: 0.9794
Epoch 122/1000
2400/2400 [==============================] - 0s 35us/step - loss: 0.0063 - acc: 0.9988 - val_loss: 0.0858 - val_acc: 0.9794
Epoch 123/1000
2400/2400 [==============================] - 0s 45us/step - loss: 0.0078 - acc: 0.9983 - val_loss: 0.0855 - val_acc: 0.9800
Epoch 124/1000
2400/2400 [==============================] - 0s 55us/step - loss: 0.0051 - acc: 0.9996 - val_loss: 0.0886 - val_acc: 0.9789
Epoch 125/1000
2400/2400 [==============================] - 0s 53us/step - loss: 0.0053 - acc: 0.9996 - val_loss: 0.0866 - val_acc: 0.9794
Epoch 126/1000
2400/2400 [================

2400/2400 [==============================] - 0s 47us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0874 - val_acc: 0.9822
Epoch 179/1000
2400/2400 [==============================] - 0s 43us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0869 - val_acc: 0.9811
Epoch 180/1000
2400/2400 [==============================] - 0s 62us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0877 - val_acc: 0.9817
Epoch 181/1000
2400/2400 [==============================] - 0s 57us/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.0878 - val_acc: 0.9794
Epoch 182/1000
2400/2400 [==============================] - 0s 67us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0899 - val_acc: 0.9811
Epoch 183/1000
2400/2400 [==============================] - 0s 58us/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.0865 - val_acc: 0.9794
Epoch 184/1000
2400/2400 [==============================] - 0s 68us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0893 - val_acc: 0.9806
Epoch 185/1000
2400/2400 [================

2400/2400 [==============================] - 0s 51us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0982 - val_acc: 0.9800
Epoch 238/1000
2400/2400 [==============================] - 0s 58us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0939 - val_acc: 0.9794
Epoch 239/1000
2400/2400 [==============================] - 0s 56us/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0945 - val_acc: 0.9800
Epoch 240/1000
2400/2400 [==============================] - 0s 51us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0968 - val_acc: 0.9817
Epoch 241/1000
2400/2400 [==============================] - 0s 62us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0977 - val_acc: 0.9806
Epoch 242/1000
2400/2400 [==============================] - 0s 67us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0956 - val_acc: 0.9806
Epoch 243/1000
2400/2400 [==============================] - 0s 67us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0970 - val_acc: 0.9794
Epoch 244/1000
2400/2400 [================

...

2400/2400 [==============================] - 0s 47us/step - loss: 9.6164e-04 - acc: 0.9996 - val_loss: 0.1203 - val_acc: 0.9794
Epoch 754/1000
2400/2400 [==============================] - 0s 60us/step - loss: 2.9995e-04 - acc: 1.0000 - val_loss: 0.1188 - val_acc: 0.9800
Epoch 755/1000
2400/2400 [==============================] - 0s 60us/step - loss: 3.5998e-04 - acc: 1.0000 - val_loss: 0.1182 - val_acc: 0.9789
Epoch 756/1000
2400/2400 [==============================] - 0s 57us/step - loss: 1.6135e-04 - acc: 1.0000 - val_loss: 0.1183 - val_acc: 0.9789
Epoch 757/1000
2400/2400 [==============================] - 0s 71us/step - loss: 3.1628e-04 - acc: 1.0000 - val_loss: 0.1178 - val_acc: 0.9794
Epoch 758/1000
2400/2400 [==============================] - 0s 60us/step - loss: 2.2035e-04 - acc: 1.0000 - val_loss: 0.1171 - val_acc: 0.9789
Epoch 759/1000
2400/2400 [==============================] - 0s 61us/step - loss: 2.2771e-04 - acc: 1.0000 - val_loss: 0.1175 - val_acc: 0.9789
Epoch 760/1000

2400/2400 [==============================] - 0s 47us/step - loss: 1.9767e-04 - acc: 1.0000 - val_loss: 0.1164 - val_acc: 0.9806
Epoch 811/1000
2400/2400 [==============================] - 0s 50us/step - loss: 1.9977e-04 - acc: 1.0000 - val_loss: 0.1173 - val_acc: 0.9794
Epoch 812/1000
2400/2400 [==============================] - 0s 54us/step - loss: 1.5387e-04 - acc: 1.0000 - val_loss: 0.1178 - val_acc: 0.9794
Epoch 813/1000
2400/2400 [==============================] - 0s 54us/step - loss: 3.0471e-04 - acc: 1.0000 - val_loss: 0.1169 - val_acc: 0.9806
Epoch 814/1000
2400/2400 [==============================] - 0s 56us/step - loss: 1.5274e-04 - acc: 1.0000 - val_loss: 0.1169 - val_acc: 0.9806
Epoch 815/1000
2400/2400 [==============================] - 0s 60us/step - loss: 2.5462e-04 - acc: 1.0000 - val_loss: 0.1181 - val_acc: 0.9794
Epoch 816/1000
2400/2400 [==============================] - 0s 55us/step - loss: 3.8171e-04 - acc: 1.0000 - val_loss: 0.1193 - val_acc: 0.9794
Epoch 817/1000

2400/2400 [==============================] - 0s 53us/step - loss: 1.8302e-04 - acc: 1.0000 - val_loss: 0.1198 - val_acc: 0.9800
Epoch 868/1000
2400/2400 [==============================] - 0s 73us/step - loss: 1.6555e-04 - acc: 1.0000 - val_loss: 0.1199 - val_acc: 0.9794
Epoch 869/1000
2400/2400 [==============================] - 0s 65us/step - loss: 1.5062e-04 - acc: 1.0000 - val_loss: 0.1199 - val_acc: 0.9794
Epoch 870/1000
2400/2400 [==============================] - 0s 59us/step - loss: 1.6516e-04 - acc: 1.0000 - val_loss: 0.1196 - val_acc: 0.9800
Epoch 871/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.7519e-04 - acc: 1.0000 - val_loss: 0.1198 - val_acc: 0.9800
Epoch 872/1000
2400/2400 [==============================] - 0s 71us/step - loss: 2.1374e-04 - acc: 1.0000 - val_loss: 0.1197 - val_acc: 0.9789
Epoch 873/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.4320e-04 - acc: 1.0000 - val_loss: 0.1194 - val_acc: 0.9789
Epoch 874/1000

2400/2400 [==============================] - 0s 44us/step - loss: 2.5054e-04 - acc: 1.0000 - val_loss: 0.1180 - val_acc: 0.9794
Epoch 925/1000
2400/2400 [==============================] - 0s 49us/step - loss: 2.1055e-04 - acc: 1.0000 - val_loss: 0.1187 - val_acc: 0.9789
Epoch 926/1000
2400/2400 [==============================] - 0s 66us/step - loss: 1.4794e-04 - acc: 1.0000 - val_loss: 0.1193 - val_acc: 0.9794
Epoch 927/1000
2400/2400 [==============================] - 0s 67us/step - loss: 1.6781e-04 - acc: 1.0000 - val_loss: 0.1194 - val_acc: 0.9794
Epoch 928/1000
2400/2400 [==============================] - 0s 58us/step - loss: 1.3446e-04 - acc: 1.0000 - val_loss: 0.1190 - val_acc: 0.9789
Epoch 929/1000
2400/2400 [==============================] - 0s 64us/step - loss: 2.2234e-04 - acc: 1.0000 - val_loss: 0.1186 - val_acc: 0.9794
Epoch 930/1000
2400/2400 [==============================] - 0s 49us/step - loss: 1.9484e-04 - acc: 1.0000 - val_loss: 0.1194 - val_acc: 0.9794
Epoch 931/1000

2400/2400 [==============================] - 0s 44us/step - loss: 1.5673e-04 - acc: 1.0000 - val_loss: 0.1208 - val_acc: 0.9794
Epoch 982/1000
2400/2400 [==============================] - 0s 51us/step - loss: 1.9659e-04 - acc: 1.0000 - val_loss: 0.1201 - val_acc: 0.9800
Epoch 983/1000
2400/2400 [==============================] - 0s 77us/step - loss: 1.8055e-04 - acc: 1.0000 - val_loss: 0.1202 - val_acc: 0.9800
Epoch 984/1000
2400/2400 [==============================] - 0s 60us/step - loss: 1.6698e-04 - acc: 1.0000 - val_loss: 0.1194 - val_acc: 0.9806
Epoch 985/1000
2400/2400 [==============================] - 0s 63us/step - loss: 1.4333e-04 - acc: 1.0000 - val_loss: 0.1191 - val_acc: 0.9806
Epoch 986/1000
2400/2400 [==============================] - 0s 51us/step - loss: 2.2999e-04 - acc: 1.0000 - val_loss: 0.1203 - val_acc: 0.9800
Epoch 987/1000
2400/2400 [==============================] - 0s 70us/step - loss: 9.8221e-05 - acc: 1.0000 - val_loss: 0.1203 - val_acc: 0.9800
Epoch 988/1000

In [ ]:
circle = []
circle_filename = []
square = []
square_filename = []
triangle = []
triangle_filename = []
horizontal = []
horizontal_filename = []
vertical = []
vertical_filename = []
diagonal = []
diagonal_filename = []

In [ ]:
for i in range(len(y_test1)):
    if y_test1[i] == 0:
        circle.append(x_test[i])
        circle_filename.append(test_labels_with_num[i])
    elif y_test1[i] == 1:
        square.append(x_test[i])
        square_filename.append(test_labels_with_num[i])
    elif y_test1[i] == 2:
        triangle.append(x_test[i])
        triangle_filename.append(test_labels_with_num[i])
    elif y_test1[i] == 3:
        horizontal.append(x_test[i])
        horizontal_filename.append(test_labels_with_num[i])
    elif y_test1[i] == 4:
        vertical.append(x_test[i])
        vertical_filename.append(test_labels_with_num[i])
    else:
        diagonal.append(x_test[i])
        diagonal_filename.append(test_labels_with_num[i])

In [ ]:
get_layer_dense_model_output = K.function([dense_model.layers[0].input], [dense_model.layers[15].output])
get_layer_gap_model_output = K.function([gap_model.layers[0].input], [gap_model.layers[13].output])

In [ ]:
get_layer_cnnAuto_gap_model_output = K.function([cnnAuto_gap_model.layers[0].input], [cnnAuto_gap_model.layers[12].output])
#get_layer_cnnAuto_gmp_model_output = K.function([cnnAuto_model.layers[0].input], [cnnAuto_gmp_model])
get_layer_cnnAuto_dense_model_output = K.function([cnnAuto_dense_model.layers[0].input], [cnnAuto_dense_model.layers[15].output])

In [ ]:
def write_output(filename, shape, size, model_type):
    result = []
    for i in range(len(shape)):
        tc = shape[i].reshape(1, img_rows, img_cols, 1)
        if model_type == 'dense':
            output = get_layer_dense_model_output([tc])
        elif model_type == 'gap':
            output = get_layer_gap_model_output([tc])
        elif model_type == 'cnnAuto_gap':
            output = get_layer_cnnAuto_gap_model_output([tc])
        elif model_type == 'cnnAuto_dense':
            output = get_layer_cnnAuto_dense_model_output([tc])
        output = np.array(output).reshape(size)
        result.append(output)
    np.save(filename, result)

In [ ]:
output_size = 64
write_output("./2nd_result/circle_dense_output", circle, output_size, 'dense')
write_output("./2nd_result/square_dense_output", square, output_size, 'dense')
write_output("./2nd_result/triangle_dense_output", triangle, output_size, 'dense')
write_output("./2nd_result/horizontal_dense_output", horizontal, output_size, 'dense')
write_output("./2nd_result/vertical_dense_output", vertical, output_size, 'dense')
write_output("./2nd_result/diagonal_dense_output", diagonal, output_size, 'dense')

write_output("./2nd_result/circle_gap_output", circle, output_size, 'gap')
write_output("./2nd_result/square_gap_output", square, output_size, 'gap')
write_output("./2nd_result/triangle_gap_output", triangle, output_size, 'gap')
write_output("./2nd_result/horizontal_gap_output", horizontal, output_size, 'gap')
write_output("./2nd_result/vertical_gap_output", vertical, output_size, 'gap')
write_output("./2nd_result/diagonal_gap_output", diagonal, output_size, 'gap')

In [ ]:
write_output("./2nd_result/circle_cnnAuto_gap_output", circle, output_size, 'cnnAuto_gap')
write_output("./2nd_result/square_cnnAuto_gap_output", square, output_size, 'cnnAuto_gap')
write_output("./2nd_result/triangle_cnnAuto_gap_output", triangle, output_size, 'cnnAuto_gap')
write_output("./2nd_result/horizontal_cnnAuto_gap_output", horizontal, output_size, 'cnnAuto_gap')
write_output("./2nd_result/vertical_cnnAuto_gap_output", vertical, output_size, 'cnnAuto_gap')
write_output("./2nd_result/diagonal_cnnAuto_gap_output", diagonal, output_size, 'cnnAuto_gap')

write_output("./2nd_result/circle_cnnAuto_dense_output", circle, output_size, 'cnnAuto_dense')
write_output("./2nd_result/square_cnnAuto_dense_output", square, output_size, 'cnnAuto_dense')
write_output("./2nd_result/triangle_cnnAuto_dense_output", triangle, output_size, 'cnnAuto_dense')
write_output("./2nd_result/horizontal_cnnAuto_dense_output", horizontal, output_size, 'cnnAuto_dense')
write_output("./2nd_result/vertical_cnnAuto_dense_output", vertical, output_size, 'cnnAuto_dense')
write_output("./2nd_result/diagonal_cnnAuto_dense_output", diagonal, output_size, 'cnnAuto_dense')

In [ ]:
total_filename = circle_filename+ square_filename + triangle_filename + horizontal_filename + vertical_filename + diagonal_filename

In [ ]:
def getMaxMin(shape):
    mmax = -1000
    mmin = 1000
    for i in range(len(shape)):
        max = -1000
        min = 1000
        for j in range(len(shape[i])):
            if max < shape[i][j]:
                max = shape[i][j]
            if min > shape[i][j]:
                min = shape[i][j]
        if mmax < max:
            mmax = max
        if mmin > min:
            mmin = min
    return mmax, mmin

In [ ]:
def read_output(model_type):
    parent_dir = './2nd_result/'
    result = []
    for i in range(len(label)):
        filename = parent_dir + label[i] + '_' + model_type + '_output.npy'
        result.append(np.load(filename))
    return result

dense_output = read_output('dense')
gap_output = read_output('gap')
cnnAuto_gap_output = read_output('cnnAuto_gap')
cnnAuto_dense_output = read_output('cnnAuto_dense')

In [ ]:
for i in range(len(label)):
    print(label[i], 'dense', getMaxMin(dense_output[i]))
print()
for i in range(len(label)):
    print(label[i], 'gap', getMaxMin(gap_output[i]))
print()
for i in range(len(label)):
    print(label[i], 'cnnAuto_gap', getMaxMin(cnnAuto_gap_output[i]))
print()
for i in range(len(label)):
    print(label[i], 'cnnAuto_dense', getMaxMin(cnnAuto_dense_output[i]))

In [ ]:
def get_total_output(model_type):
    if model_type == 'dense':
        model = dense_output
    elif model_type == 'gap':
        model = gap_output
    elif model_type == 'cnnAuto_gap':
        model = cnnAuto_gap_output
    elif model_type == 'cnnAuto_dense':
        model = cnnAuto_dense_output
    total_output = model[0]
    for i in range(1, len(label)):
        total_output = np.vstack([total_output, model[i]])
    return total_output

In [ ]:
def getCountTop(total, ordered_shape_dist, shape):
    count = [0, 0, 0, 0, 0, 0]
    for i in range(0, total):
        for j in range(0, len(shape)):
            if(total_filename[ordered_shape_dist[i]].find(shape[j]) != -1):
                count[j] = count[j] + 1
                break
    return count

In [ ]:
def rmFileWithTC(testcase):
    dirname = os.getcwd() + '/2nd_result/tc'+testcase+"/"
    files = os.listdir(dirname)
    for f in files:
        if 'tc' in f and '.png' in f and 'test' in f:
            os.remove(dirname +f)

In [ ]:
def getCompareImages(shape, ordered_shape_dist, testcase, model_type):
    
    path = './dataset/tests/'
    images = []
    for i in range(len(ordered_shape_dist)):
        images.append(path + total_filename[ordered_shape_dist[i]] + ".png")
    
    names = []
    unit = 30
    dirname = './2nd_result/tc' + testcase
    if  not os.path.isdir(dirname) :
        os.mkdir(dirname)
    
    for i in range(0, int(len(ordered_shape_dist)/unit)):
        imgs = [PIL.Image.open(i).convert('L') for i in images[i*unit:unit*(i+1)]]
        min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
        imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
        imgs_comb = PIL.Image.fromarray(imgs_comb)
        filename = dirname + "/tc"+testcase+"_test_"+ model_type + "_" + shape + str(i) + ".png"
        names.append(filename)
        imgs_comb.save(filename)

    imgs = [PIL.Image.open(i).convert('L') for i in names]
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
    imgs_comb = PIL.Image.fromarray( imgs_comb)
    imgs_comb.save( dirname+ '/tc' + testcase+'_' + model_type + "_" + shape+ '.png' )
    
    rmFileWithTC(testcase)
    
    display(Image(path+total_filename[ordered_shape_dist[0]]+".png"))
    display(Image(dirname + '/tc'+testcase+'_'+model_type + "_"+shape+'.png' ))

In [ ]:
def getImages(model_type, tc):
    if model_type == 'dense':
        getCompareImages("circle", ordered_circle_dense_dist, tc, model_type)
        getCompareImages("square", ordered_square_dense_dist, tc, model_type)
        getCompareImages("triangle", ordered_triangle_dense_dist, tc, model_type)
        getCompareImages("horizontal", ordered_hor_dense_dist, tc, model_type)
        getCompareImages("vertical", ordered_ver_dense_dist, tc, model_type)
        getCompareImages("diagonal", ordered_dia_dense_dist, tc, model_type)
    elif model_type == 'gap':
        getCompareImages("circle", ordered_circle_gap_dist, tc, model_type)
        getCompareImages("square", ordered_square_gap_dist, tc, model_type)
        getCompareImages("triangle", ordered_triangle_gap_dist, tc, model_type)
        getCompareImages("horizontal", ordered_hor_gap_dist, tc, model_type)
        getCompareImages("vertical", ordered_ver_gap_dist, tc, model_type)
        getCompareImages("diagonal", ordered_dia_gap_dist, tc, model_type)
    elif model_type == 'cnnAuto_gap':
        getCompareImages("circle", ordered_circle_cnnAuto_gap_dist, tc, model_type)
        getCompareImages("square", ordered_square_cnnAuto_gap_dist, tc, model_type)
        getCompareImages("triangle", ordered_triangle_cnnAuto_gap_dist, tc, model_type)
        getCompareImages("horizontal", ordered_hor_cnnAuto_gap_dist, tc, model_type)
        getCompareImages("vertical", ordered_ver_cnnAuto_gap_dist, tc, model_type)
        getCompareImages("diagonal", ordered_dia_cnnAuto_gap_dist, tc, model_type)
    elif model_type == 'cnnAuto_dense':
        getCompareImages("circle", ordered_circle_cnnAuto_dense_dist, tc, model_type)
        getCompareImages("square", ordered_square_cnnAuto_dense_dist, tc, model_type)
        getCompareImages("triangle", ordered_triangle_cnnAuto_dense_dist, tc, model_type)
        getCompareImages("horizontal", ordered_hor_cnnAuto_dense_dist, tc, model_type)
        getCompareImages("vertical", ordered_ver_cnnAuto_dense_dist, tc, model_type)
        getCompareImages("diagonal", ordered_dia_cnnAuto_dense_dist, tc, model_type)

In [ ]:
dense_distribution = np.array([[0 for i in range(6)] for j in range(6)])
gap_distribution = np.array([[0 for i in range(6)] for j in range(6)])
cnnAuto_gap_distribution = np.array([[0 for i in range(6)] for j in range(6)])
cnnAuto_dense_distribution = np.array([[0 for i in range(6)] for j in range(6)])

In [ ]:
testCases = 30.

# category 별 test set 갯수 200
for i in range(int(testCases)):
    rand_index = random.randint(0, 199)
    
    total_dense = get_total_output('dense')
    total_gap = get_total_output('gap')
    total_cnnAuto_gap = get_total_output('cnnAuto_gap')
    total_cnnAuto_dense = get_total_output('cnnAuto_dense')
        
    circle_dense_dist = []
    square_dense_dist = []
    triangle_dense_dist = []
    hor_dense_dist = []
    ver_dense_dist = []
    dia_dense_dist = []
    
    circle_gap_dist = []
    square_gap_dist = []
    triangle_gap_dist = []
    hor_gap_dist = []
    ver_gap_dist = []
    dia_gap_dist = []

    circle_cnnAuto_gap_dist = []
    square_cnnAuto_gap_dist = []
    triangle_cnnAuto_gap_dist = []
    hor_cnnAuto_gap_dist = []
    ver_cnnAuto_gap_dist = []
    dia_cnnAuto_gap_dist = []
    
    circle_cnnAuto_dense_dist = []
    square_cnnAuto_dense_dist = []
    triangle_cnnAuto_dense_dist = []
    hor_cnnAuto_dense_dist = []
    ver_cnnAuto_dense_dist = []
    dia_cnnAuto_dense_dist = []
    
    for line in total_dense:
        '''    #(total_dense[rand_index], line)
        circle_dense_dist.append(cosine_similarity(total_dense[rand_index].reshape(1, -1), line.reshape(1, -1)).reshape(1, )[0])
        square_dense_dist.append(cosine_similarity(total_dense[200+rand_index].reshape(1, -1), line.reshape(1, -1)).reshape(1, )[0])
        triangle_dense_dist.append(cosine_similarity(total_dense[400+rand_index].reshape(1, -1), line.reshape(1, -1)).reshape(1, )[0])
        hor_dense_dist.append(cosine_similarity(total_dense[600+rand_index].reshape(1, -1), line.reshape(1, -1)).reshape(1, )[0])
        ver_dense_dist.append(cosine_similarity(total_dense[800+rand_index].reshape(1, -1), line.reshape(1, -1)).reshape(1, )[0])
        dia_dense_dist.append(cosine_similarity(total_dense[1000+rand_index].reshape(1, -1), line.reshape(1, -1)).reshape(1, )[0])
        '''
        circle_dense_dist.append(distance.euclidean(total_dense[rand_index], line))
        square_dense_dist.append(distance.euclidean(total_dense[200+rand_index], line))
        triangle_dense_dist.append(distance.euclidean(total_dense[400+rand_index], line))
        hor_dense_dist.append(distance.euclidean(total_dense[600+rand_index], line))
        ver_dense_dist.append(distance.euclidean(total_dense[800+rand_index], line))
        dia_dense_dist.append(distance.euclidean(total_dense[1000+rand_index], line))
    
    for line in total_gap:
        circle_gap_dist.append(distance.euclidean(total_gap[rand_index], line))
        square_gap_dist.append(distance.euclidean(total_gap[200+rand_index], line))
        triangle_gap_dist.append(distance.euclidean(total_gap[400+rand_index], line))
        hor_gap_dist.append(distance.euclidean(total_gap[600+rand_index], line))
        ver_gap_dist.append(distance.euclidean(total_gap[800+rand_index], line))
        dia_gap_dist.append(distance.euclidean(total_gap[1000+rand_index], line))
    
    for line in total_cnnAuto_gap:
        circle_cnnAuto_gap_dist.append(distance.euclidean(total_cnnAuto_gap[rand_index], line))
        square_cnnAuto_gap_dist.append(distance.euclidean(total_cnnAuto_gap[200+rand_index], line))
        triangle_cnnAuto_gap_dist.append(distance.euclidean(total_cnnAuto_gap[400+rand_index], line))
        hor_cnnAuto_gap_dist.append(distance.euclidean(total_cnnAuto_gap[600+rand_index], line))
        ver_cnnAuto_gap_dist.append(distance.euclidean(total_cnnAuto_gap[800+rand_index], line))
        dia_cnnAuto_gap_dist.append(distance.euclidean(total_cnnAuto_gap[1000+rand_index], line))
    
    for line in total_cnnAuto_dense:
        circle_cnnAuto_dense_dist.append(distance.euclidean(total_cnnAuto_dense[rand_index], line))
        square_cnnAuto_dense_dist.append(distance.euclidean(total_cnnAuto_dense[200+rand_index], line))
        triangle_cnnAuto_dense_dist.append(distance.euclidean(total_cnnAuto_dense[400+rand_index], line))
        hor_cnnAuto_dense_dist.append(distance.euclidean(total_cnnAuto_dense[600+rand_index], line))
        ver_cnnAuto_dense_dist.append(distance.euclidean(total_cnnAuto_dense[800+rand_index], line))
        dia_cnnAuto_dense_dist.append(distance.euclidean(total_cnnAuto_dense[1000+rand_index], line))
    
    ordered_circle_dense_dist = np.array(circle_dense_dist).argsort()
    #ordered_circle_dense_dist = ordered_circle_dense_dist[::-1]
    ordered_square_dense_dist = np.array(square_dense_dist).argsort()
    ordered_triangle_dense_dist = np.array(triangle_dense_dist).argsort()
    ordered_hor_dense_dist = np.array(hor_dense_dist).argsort()
    ordered_ver_dense_dist = np.array(ver_dense_dist).argsort()
    ordered_dia_dense_dist = np.array(dia_dense_dist).argsort()

    ordered_circle_gap_dist = np.array(circle_gap_dist).argsort()
    ordered_square_gap_dist = np.array(square_gap_dist).argsort()
    ordered_triangle_gap_dist = np.array(triangle_gap_dist).argsort()
    ordered_hor_gap_dist = np.array(hor_gap_dist).argsort()
    ordered_ver_gap_dist = np.array(ver_gap_dist).argsort()
    ordered_dia_gap_dist = np.array(dia_gap_dist).argsort()
    
    ordered_circle_cnnAuto_gap_dist = np.array(circle_cnnAuto_gap_dist).argsort()
    ordered_square_cnnAuto_gap_dist = np.array(square_cnnAuto_gap_dist).argsort()
    ordered_triangle_cnnAuto_gap_dist = np.array(triangle_cnnAuto_gap_dist).argsort()
    ordered_hor_cnnAuto_gap_dist = np.array(hor_cnnAuto_gap_dist).argsort()
    ordered_ver_cnnAuto_gap_dist = np.array(ver_cnnAuto_gap_dist).argsort()
    ordered_dia_cnnAuto_gap_dist = np.array(dia_cnnAuto_gap_dist).argsort()
    
    ordered_circle_cnnAuto_dense_dist = np.array(circle_cnnAuto_dense_dist).argsort()
    ordered_square_cnnAuto_dense_dist = np.array(square_cnnAuto_dense_dist).argsort()
    ordered_triangle_cnnAuto_dense_dist = np.array(triangle_cnnAuto_dense_dist).argsort()
    ordered_hor_cnnAuto_dense_dist = np.array(hor_cnnAuto_dense_dist).argsort()
    ordered_ver_cnnAuto_dense_dist = np.array(ver_cnnAuto_dense_dist).argsort()
    ordered_dia_cnnAuto_dense_dist = np.array(dia_cnnAuto_dense_dist).argsort()
    
    topNumber = 100
    
    shape_dense_distribution = []
    shape_dense_distribution.append(getCountTop(topNumber, ordered_circle_dense_dist, label))
    shape_dense_distribution.append(getCountTop(topNumber, ordered_square_dense_dist, label))
    shape_dense_distribution.append(getCountTop(topNumber, ordered_triangle_dense_dist, label))
    shape_dense_distribution.append(getCountTop(topNumber, ordered_hor_dense_dist, label))
    shape_dense_distribution.append(getCountTop(topNumber, ordered_ver_dense_dist, label))
    shape_dense_distribution.append(getCountTop(topNumber, ordered_dia_dense_dist, label))
    
    shape_gap_distribution = []
    shape_gap_distribution.append(getCountTop(topNumber, ordered_circle_gap_dist, label))
    shape_gap_distribution.append(getCountTop(topNumber, ordered_square_gap_dist, label))
    shape_gap_distribution.append(getCountTop(topNumber, ordered_triangle_gap_dist, label))
    shape_gap_distribution.append(getCountTop(topNumber, ordered_hor_gap_dist, label))
    shape_gap_distribution.append(getCountTop(topNumber, ordered_ver_gap_dist, label))
    shape_gap_distribution.append(getCountTop(topNumber, ordered_dia_gap_dist, label))
    
    shape_cnnAuto_gap_distribution = []
    shape_cnnAuto_gap_distribution.append(getCountTop(topNumber, ordered_circle_cnnAuto_gap_dist, label))
    shape_cnnAuto_gap_distribution.append(getCountTop(topNumber, ordered_square_cnnAuto_gap_dist, label))
    shape_cnnAuto_gap_distribution.append(getCountTop(topNumber, ordered_triangle_cnnAuto_gap_dist, label))
    shape_cnnAuto_gap_distribution.append(getCountTop(topNumber, ordered_hor_cnnAuto_gap_dist, label))
    shape_cnnAuto_gap_distribution.append(getCountTop(topNumber, ordered_ver_cnnAuto_gap_dist, label))
    shape_cnnAuto_gap_distribution.append(getCountTop(topNumber, ordered_dia_cnnAuto_gap_dist, label))
    
    shape_cnnAuto_dense_distribution = []
    shape_cnnAuto_dense_distribution.append(getCountTop(topNumber, ordered_circle_cnnAuto_dense_dist, label))
    shape_cnnAuto_dense_distribution.append(getCountTop(topNumber, ordered_square_cnnAuto_dense_dist, label))
    shape_cnnAuto_dense_distribution.append(getCountTop(topNumber, ordered_triangle_cnnAuto_dense_dist, label))
    shape_cnnAuto_dense_distribution.append(getCountTop(topNumber, ordered_hor_cnnAuto_dense_dist, label))
    shape_cnnAuto_dense_distribution.append(getCountTop(topNumber, ordered_ver_cnnAuto_dense_dist, label))
    shape_cnnAuto_dense_distribution.append(getCountTop(topNumber, ordered_dia_cnnAuto_dense_dist, label))
    
    if i < 5:
        print("# dense model prediction")
        getImages('dense', str(i+1))
        print("# gap model prediction")
        getImages('gap', str(i+1))
        print("# cnnAuto gap model prediction")
        getImages('cnnAuto_gap', str(i+1))
        print("# cnnAuto dense model prediction")
        getImages('cnnAuto_dense', str(i+1))
        
    for i in range(0, len(label)):
        print(label[i], "dense distribution :", shape_dense_distribution[i])
        dense_distribution[i] = dense_distribution[i] + shape_dense_distribution[i]

    print()
    for i in range(0, len(label)):
        print(label[i], "gap distribution :", shape_gap_distribution[i])
        gap_distribution[i] = gap_distribution[i] + shape_gap_distribution[i]
    print()
    for i in range(0, len(label)):
        print(label[i], "cnnAuto gap distribution :", shape_cnnAuto_gap_distribution[i])
        cnnAuto_gap_distribution[i] = cnnAuto_gap_distribution[i] + shape_cnnAuto_gap_distribution[i]
        
    print()
    for i in range(0, len(label)):
        print(label[i], "cnnAuto dense distribution :", shape_cnnAuto_dense_distribution[i])
        cnnAuto_dense_distribution[i] = cnnAuto_dense_distribution[i] + shape_cnnAuto_dense_distribution[i]

In [ ]:
dense_avg = dense_distribution/testCases
gap_avg = gap_distribution/testCases
cnnAuto_gap_avg = cnnAuto_gap_distribution/testCases
cnnAuto_dense_avg = cnnAuto_dense_distribution/testCases

print('\nAVG all shape')
np.set_printoptions(suppress=True)
print('Dense: \n', dense_avg)
print('\nGAP: \n', gap_avg)
print('\ncnnAuto gap: \n', cnnAuto_gap_avg)
print('\ncnnAuto dense: \n', cnnAuto_dense_avg)